# 「ヒューマノイドロボット」の本に記載されているプログラムをPythonで動かす！

「<a href="https://www.amazon.co.jp/%E3%83%92%E3%83%A5%E3%83%BC%E3%83%9E%E3%83%8E%E3%82%A4%E3%83%89%E3%83%AD%E3%83%9C%E3%83%83%E3%83%88-%E6%94%B9%E8%A8%822%E7%89%88-%E6%A2%B6%E7%94%B0-%E7%A7%80%E5%8F%B8/dp/4274226026/ref=sr_1_2?crid=2DGXE43QVGNMF&keywords=%E3%83%92%E3%83%A5%E3%83%BC%E3%83%9E%E3%83%8E%E3%82%A4%E3%83%89&qid=1645525881&sprefix=%E3%83%92%E3%83%A5%E3%83%BC%E3%83%9E%E3%83%8E%E3%82%A4%E3%83%89%2Caps%2C173&sr=8-2">ヒューマノイドロボット</a>(著：梶田修司，オーム社)」の本に記載されているプログラムは<a href="https://github.com/s-kajita/IntroductionToHumanoidRobotics">Matlab形式</a>で，無料のOctaveでも実行が可能ですが，Pythonのプログラムでも確認できるようにできないかと考えてPythonの練習も兼ねて移植してみました．

一応，最後の2足歩行ロボットが崩れ落ちる動力学まで動くようになりました．Octaveで実行したよりPythonのほうが速く動いています．（2022年2月22日ノートパソコンMSI GS65-8RFで確認）

もっと速くするために，余裕があればC++に移植したい．（Choreonoid使えばいいだけですが．．．）

今後の予定<br>
・ <a href="http://royfeatherstone.org/spatial/">Feather Stone</a>先生のアルゴリズムを移植してみる？<br>
・ C++に変換してみる．<br>

## １．データ構造の定義
ヒューマノイドロボットの本と同じくpythonのclassでuLINKの構造を定義する．

下記のデータ構造の例は，「ヒューマノイドロボット」の第２章の例と同じく「name（名前）」，「sister（リンクの兄弟（姉妹））」，「child（リンク子供）」，「m（リンクの質量）」の他に，「mother（リンクの親）」を加えている．


In [ ]:
class uLINK:
  def __init__(self, name, sister, child, m, mother):
    self.name = name
    self.sister = sister
    self.child = child
    self.m = m
    self.mother = mother

上記のクラスを使ってリスト構造にデータを代入する．

初期化では，２分木の構造の兄弟（姉妹）と子供のリンク構造だけ与えておき，リンクの親についてはゼロを代入している．

親リンク(mother)については，後にあるFindmotherの関数で自動的に代入されるようになっている．

In [ ]:
arr=[0]*10 #10個のリストをゼロで初期化
arr[0]=uLINK("empty",0,0,0,0) #uLINKクラスの定義に沿ってデータを代入
arr[1]=uLINK("BODY",0,2,10,0)
arr[2]=uLINK("RARM",4,3,5,0)
arr[3]=uLINK("RHAND",0,0,1,0)
arr[4]=uLINK("LARM",6,5,5,0)
arr[5]=uLINK("LHAND",0,0,1,0)
arr[6]=uLINK("RLEG",8,7,6,0)
arr[7]=uLINK("RFOOT",0,0,2,0)
arr[8]=uLINK("LLEG",0,9,6,0)
arr[9]=uLINK("LFOOT",0,0,2,0)

### FindMther関数

兄弟と子供のリンク構造を定義しておけば，この関数を使うことで，リンクの親の情報を求めることができる．

In [ ]:
def FindMother(j):
    if j != 0:
        if j == 1:
            arr[j].mother = 0;
        if arr[j].child != 0:
            arr[arr[j].child].mother = j;
            FindMother(arr[j].child);
        if arr[j].sister != 0:
            arr[arr[j].sister].mother = arr[j].mother;
            FindMother(arr[j].sister);

下記は，FindMother関数の実行例

FindMother関数の引数に与えたリンク番号の下に位置している兄弟リンクと子供リンクのすべての親のIDをリンク構造の定義motherに代入してくれる．

In [ ]:
FindMother(1) #ID1以下のリンクの親のIDを登録
print("arr[8].mother: ", arr[8].mother) #リンク8の親のIDを表示

### TotalMass関数

引数に与えたリンク番号以下のリンク構造すべての質量の和を計算する．

In [ ]:
def TotalMass(x):
  if x == 0:
    m = 0
    return m
  else:
    m = arr[x].m + TotalMass(arr[x].sister) + TotalMass(arr[x].child)
    return m

In [ ]:
TotalMass(1) #リンク１以下のすべての質量の和．

### PrintLinkName関数

引数に与えたリンク番号以下のリンク番号と名前を表示する関数．再帰関数を使って自分のリンクの子供と兄弟のリンクの名前を呼び出すことで指定したリンク以下の名前が表示されるようになっている．ただし，最初のIDの付け方を間違えると無限ループするので注意が必要．この関数を実行することで，きちんとリンクのつながりを定義できているか確認することができる．

In [ ]:
def PrintLinkName(j):
    if j != 0:
        print('ID=', j, arr[j].name)
        PrintLinkName(arr[j].child);
        PrintLinkName(arr[j].sister);
    

In [ ]:
PrintLinkName(1) #リンク１以下のすべてのリンク番号と名前を表示

### Rodriges関数

ロドリゲスの式を使った回転行列の計算を行うための関数．
入力は，第１引数に回転ベクトル（回転軸を表す単位ベクトル），第２引数に回転軸周りに回転した角度を与えることで，回転行列を得ることができる．

In [ ]:
def Rodrigues(w,dt):
    norm_w = np.linalg.norm(w);
    if norm_w < sys.float_info.epsilon :
        R = np.eye(3);
    else :
        th = norm_w * dt;
        wn = w/norm_w;
        w_wedge = np.array([ [0, -wn[2], wn[1]],[wn[2], 0, -wn[0]],[-wn[1], wn[0], 0]]);
        R = np.eye(3) + w_wedge * math.sin(th) + np.matmul(w_wedge,w_wedge) * (1-math.cos(th));
    return R

In [ ]:
#Rodrigues関数の利用例
import numpy as np #行列計算のためnumpyをインポート
import math        #三角関数を使うためにmathをインポート
import sys         #計算機イプシロンを使うためにsysをインポート
w=[1.0, 0.0, 0.0]; #回転ベクトルを定義：x軸回りの回転を表す
dt=0.5;            #回転ベクトル周りの回転量(rad)を与える
Rodrigues(w,dt)    #回転行列R(3x3行列)を計算

## 順運動学
これまでに定義した関数を使って再帰的に関数を呼び出すことで現在の関節角で決定される各リンクの位置を求めていく．

### ForwardKinematics関数

順運動学の計算を行う関数．引数を与えたリンク以下のすべてのリンクの関節位置を現在の関節角度から計算する．再帰関数を利用しているため非常にシンプルにすべての関節の位置と姿勢（回転行列）を得ることができる．

In [ ]:
def ForwardKinematics(j):
    if j == 0: return
    if j != 1:
        mom = arr[j].mother;
        arr[j].p = np.matmul(arr[mom].R, arr[j].b) + arr[mom].p #　絶対座標系の位置に回転行列で戻してから位置を加えていくことで絶対座標系での関節位置を求める
        ROD = Rodrigues(arr[j].a, arr[j].q); #　回転行列をロドリゲスの関数から導く
        arr[j].R = np.matmul(arr[mom].R, ROD); #　一つ手前（親）のリンクの回転行列をかけて，絶対座標系でのリンクの姿勢行列を求める
    ForwardKinematics(arr[j].sister);
    ForwardKinematics(arr[j].child);

### データ構造の定義の追加

一番初めに定義した「name（名前）」，「sister（リンクの兄弟（姉妹））」，「child（リンク子供）」，「m（リンクの質量）」の他に，「mother（リンクの親）」の他に，順運動学計算に必要となるリンク間距離(b：３次元ベクトル)や関節の回転軸(a：３次元ベクトル)，関節の回転角(q：スカラ)，リンクの回転行列(R：絶対座標基準)，リンクの位置(p：絶対座標基準)を追加している．

In [ ]:
class uLINK:
  def __init__(self, name, sister, child, m, mother, b ,a ,q,R,p):
    self.name = name
    self.sister = sister
    self.child = child
    self.mother = mother
    self.m = m
    self.b = b # リンク間距離（親リンク座標系相対位置でのリンクの長さ）
    self.a = a # 関節軸の方向（親リンク座標系相対の回転ベクトル方向）
    self.q = q # 関節の回転角度
    self.R = R # リンクの姿勢行列（回転行列）：絶対座標基準
    self.p = p # リンクの関節の位置：絶対座標基準

新しく設定した定義に合わせて，リンクの情報の設定例を下記に示す．

右足(RLEG）と左足（LLEG）はそれぞれJ0からJ5の６つの関節を有しているロボットアーム（レッグ）となっている．

股関節（肩関節）に３軸直交（J0のZ軸回転（ヨー）とJ1のX軸回転（ロール），J2のY軸回転（ピッチ）が同じ位置に設定されている）となっており，膝関節はJ3関節にY軸回り（ピッチ）の関節，足首には，J4関節にY軸（ヨー）とJ5関節にX軸（ロール）の関節を与えている．


In [ ]:
ToDeg = 180/math.pi;   #r adianからdegreeに変換するときの係数
ToRad = math.pi/180;   #d egreeからradianに変換するときの係数

UX = [1.0,0.0,0.0];    # X軸の単位ベクトルで関節軸を設定するためのシンボル設定
UY = [0.0,1.0,0.0];    # Y軸の単位ベクトルで関節軸を設定するためのシンボル設定
UZ = [0.0,0.0,1.0];    # Z軸の単位ベクトルで関節軸を設定するためのシンボル設定

R0=np.eye(3);          # 単位行列のシンボル設定
p0= [0,0,0];           # ゼロ位置のシンボル設定

arr = [0]*14          # リンクの数（この例では14リンク）の配列を初期化

arr[0]=uLINK("empty",0,0,0,0,0,0,0,R0,p0) # 空のゼロリンクを設定（ワールド座標系）
arr[1]=uLINK("BODY", 0, 2, 0, 10, [0.0, 0.0, 0.7],UZ,0,R0,p0) # Bodyのリンク初期化

# 右足のリンクの初期化：第3関節（股関節のピッチ）と第4関節（膝関節）のピッチ角を与えている）
arr[2]=uLINK("RLEG_J0", 8, 3, 0, 5, [0.0,-0.1, 0.0],UZ,0,R0,p0)
arr[3]=uLINK("RLEG_J1", 0, 4, 0, 1, [0.0, 0.0, 0.0],UX,0,R0,p0)
arr[4]=uLINK("RLEG_J2", 0, 5, 0, 5, [0.0, 0.0, 0.0],UY,-90*ToRad,R0,p0)
arr[5]=uLINK("RLEG_J3", 0, 6, 0, 1, [0.0, 0.0,-0.3],UY,90*ToRad,R0,p0)
arr[6]=uLINK("RLEG_J4", 0, 7, 0, 6, [0.0, 0.0,-0.3],UY,0,R0,p0)
arr[7]=uLINK("RLEG_J5", 0, 0, 0, 2, [0.0, 0.0, 0.0],UX,0,R0,p0)

# 左足のリンクの初期化：第10関節（股関節のピッチ）と第11関節（膝関節）のピッチ角を与えている）
arr[8] =uLINK("LLEG_J0", 0,  9, 0, 5, [0.0, 0.1, 0.0],UZ,0,R0,p0)
arr[9] =uLINK("LLEG_J1", 0, 10, 0, 1, [0.0, 0.0, 0.0],UX,0,R0,p0)
arr[10]=uLINK("LLEG_J2", 0, 11, 0, 5, [0.0, 0.0, 0.0],UY,-90*ToRad,R0,p0)
arr[11]=uLINK("LLEG_J3", 0, 12, 0, 1, [0.0, 0.0,-0.3],UY,90*ToRad,R0,p0)
arr[12]=uLINK("LLEG_J4", 0, 13, 0, 6, [0.0, 0.0,-0.3],UY,0,R0,p0)
arr[13]=uLINK("LLEG_J5", 0,  0, 0, 2, [0.0, 0.0, 0.0],UX,0,R0,p0)

FindMother(1)

上記の様に各リンクの設定が適切に行われていれば，ForwardKinematics関数を呼び出すだけで，各関節の位置と姿勢を求めることができる．

下記に使用例を示す．

In [ ]:
ForwardKinematics(1) #第１リンク以下のすべてのリンク位置と姿勢を計算
print(arr[7].p)      #第７リンクの関節位置を表示（右足首の位置）
print(arr[7].R)      #第７リンクの関節の姿勢行列を表示（右足首の姿勢行列）
print(arr[13].p)     #第13リンクの関節位置を表示（左足首の位置）
print(arr[13].R)     #第13リンクの関節の姿勢行列を表示（左足首の姿勢行列）

イメージがつきやすいように下記のコードで，3次元のグラフに各関節の位置をプロットして線でつなげた図を示す．

「#inline」をつけることで，プロット図をjupyternotebook上でも見ることができる．

下記では記載していないが「%matplotlib notebook」と記載をしておくと，3次元プロットをドラッグして別の視点からも見ることができる．

In [ ]:
# 描画の例　関節に丸を描いて，関節と関節を線で結んだだけの図
# #inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
r_leg = np.array([arr[1].p,arr[2].p,arr[3].p,arr[4].p,arr[5].p,arr[6].p,arr[7].p])
l_leg = np.array([arr[1].p,arr[8].p,arr[9].p,arr[10].p,arr[11].p,arr[12].p,arr[13].p])

ax.plot(r_leg[:,0],r_leg[:,1],r_leg[:,2],"o-", c='b') # 右足の位置を線で接続したプロット
ax.plot(l_leg[:,0],l_leg[:,1],l_leg[:,2],"o-", c='b') # 左足の位置を線で接続したプロット
ax.set_xticks(np.linspace(-1.0, 1.0, 5)) # X軸の表示範囲
ax.set_yticks(np.linspace(-1.0, 1.0, 5)) # Y軸の表示範囲
ax.set_zticks(np.linspace(-1.0, 1.0, 5)) # Z軸の表示範囲

plt.show()

### RPY2R関数

roll(X軸回りの回転角），pitch(Y軸回りの回転角），yaw(Z軸回りの回転角）の角度から回転行列R（３ｘ３）を求める関数．


In [ ]:
def RPY2R(rpy): # 入力rpyはnumpyのarrayで1行3列のベクトル
    roll = rpy[0]; pitch = rpy[1]; yaw = rpy[2];
    Cphi = math.cos(roll);  Sphi = math.sin(roll);
    Cthe = math.cos(pitch); Sthe = math.sin(pitch);
    Cpsi = math.cos(yaw);   Spsi = math.sin(yaw);

    R = [[Cpsi*Cthe,  -Spsi*Cphi+Cpsi*Sthe*Sphi,  Spsi*Sphi+Cpsi*Sthe*Cphi],
       [Spsi*Cthe,  Cpsi*Cphi+Spsi*Sthe*Sphi,   -Cpsi*Sphi+Spsi*Sthe*Cphi],
       [-Sthe,      Cthe*Sphi,                  Cthe*Cphi]                 ]
    return R


### FindRoute関数

求めたい位置・姿勢のリンクを引数に与えると，Bodyリンクからのリンクのつながり（ルート）を配列で返してくれる．

例えば，引数に右足首の最終関節のリンク番号7を与えるとボディリンク１からどの関節を経由してリンク７につながっているか配列で返してくれる．<br>
FindRoute(7)　と入力すると<br>
[2,3,4,5,6,7]　という配列が返ってくる

In [ ]:
def FindRoute(to): # 入力toには経路を知りたいリンクのIDを与える
#return the list of joint number connecting ROOT to 'to'
    i = arr[to].mother;
#if i == 0
#    idx = [];    # search failed
#    fprintf('FindRoute: search failed\n');
#else
    if i == 1:
        idx = [to];
    else:
        idx = FindRoute(i) + [to];
    return idx


In [ ]:
FindRoute(7) # 第７リンクまでの関節のつながりを表示 # BodyのID 1から[2,3,4,5,6,7]とつながっていることがわかる

## ３．逆運動学１

ヤコビ行列を使った逆運動学の導出．

### rot2omega関数

回転行列R(:３ｘ３行列)から[roll(X軸回りの回転), pitch(Y軸回りの回転), yaw(Z軸回りの回転)]の１行３列のベクトルを算出．単位はradian．

In [ ]:
def rot2omega(R):
    # T.Sugihara "Solvability-unconcerned Inverse Kinemacics based on 
    # Levenberg-Marquardt method with Robust Damping," Humanoids 2009
    #el = [R[3,2]-R[2,3], R[1,3]-R[3,1], R[2,1]-R[1,2]];
    el = np.array([R[2,1]-R[1,2], R[0,2]-R[2,0], R[1,0]-R[0,1]]);
    norm_el = np.linalg.norm(el);
    if norm_el >  sys.float_info.epsilon :
        #print("el ",el)
        #print("norm_el ",norm_el)
        #print("np.trace(R)", (np.trace(R)-1))
        #print("atan2 ",math.atan2(norm_el, np.trace(R)-1))
        w = math.atan2(norm_el, np.trace(R)-1)/norm_el * el;
    elif R[0,0]>0 and R[1,1]>0 and R[2,2]>0 :#elif R[1,1]>0 and R[2,2]>0 and R[3,3]>0 :
        w = [0, 0, 0];
    else :
        w = math.pi/2*[R(0,0)+1, R(1,1)+1, R(2,2)+1];#w = math.pi/2*[R(1,1)+1, R(2,2)+1, R(3,3)+1];
    return w # [roll, pitch, yaw]の単位はrad  

### CalcWerr関数

目標位置・姿勢と現在位置の誤差を算出する関数．

第１引数(arr1)：現在位置・姿勢の６次元ベクトル(x,y,z,roll,pitch,yaw)
第２引数(arr2)：目的位置・姿勢の６次元ベクトル(x,y,z,roll,pitch,yaw)

In [ ]:
def CalcVWerr(arr1, arr2):
    perr = arr1.p - arr2.p;
    Rerr = np.matmul(arr2.R.T, arr1.R);
    werr = np.matmul(arr2.R, rot2omega(Rerr));
    #print("rot2omega",rot2omega(Rerr))
    #err = [perr, werr];
    err = [0]*6

    for k in range(3):
        err[k] = perr[k];
        err[k+3] = werr[k];
    return err

### CalcJacobian_rot関数
回転行列だけのヤコビアンを計算する関数．使わないと思うけど一応用意．


In [ ]:
def  CalcJacobian_rot(idx) :
    #Jacobian matrix of current configration in World frame
    jsize = len(idx);
    target = arr[idx[jsize-1]].p;#   absolute target position
    J = np.zeros(3,jsize);

    for n in range(jsize):
        j = idx[n];
        a = np.matmul(arr[j].R, arr[j].a);#  joint axis vector in world frame
        J[:,n] = np.cross(a, target - arr[j].p);
    return J

### CalcJacobian関数
並進と回転のヤコビアンを計算する関数．通常はこちらだけ使う．

引数はリンクID．

引数にIDを与えると，基準のIDから引数に与えたIDまでのヤコビアンの計算をしてくれる．

In [ ]:
def CalcJacobian(idx) :
    # Jacobian matrix of current configration in World frame
    jsize = len(idx);
    target = arr[idx[jsize-1]].p;   # absolute target position
    J = np.zeros((6,jsize));
    #JT = np.zeros((jsize,6));

    for n in range(jsize) : #n=1:jsize
        j = idx[n];
        #print("j ", j)
        #print("arr.R ", arr[j].R)
        #print("arr.a ", arr[j].a)
        a = np.matmul(arr[j].R, arr[j].a);  # joint axis vector in world frame
        #print("cross ", np.cross(a, target - arr[j].p) )
        #print("a ", a)
        cross = np.cross(a, target - arr[j].p);
        #print("len(cross) ", len(cross))
        for k in range(len(cross)):
            J[k, n] = cross[k];
            J[k+len(cross),n] = a[k];

    return J



### InverseKinematics関数
ヤコビ行列を使って，現在位置と目標位置の差のある方向にニュートン法を使って少しずつ動かして目的の位置姿勢へリンク先端を動かす逆行列計算．現在位置と目標位置の差はCalcWerr関数で求める．

引数は第１引数（to）：動かすリンクID，第２引数(Target)：目標位置・姿勢（x,y,z,roll,pitch,yaw)の６次元ベクトル

In [ ]:
def InverseKinematics(to, Target):
    lam = 0.9;
    idx = FindRoute(to);
    #print("idx ", idx)
    ForwardKinematics(1);
    err = CalcVWerr(Target, arr[to]);
   # print("err ",err)
    for n in range(10):
        if np.linalg.norm(err) < 1e-6 : break;
        J  = CalcJacobian(idx);
        #print("J ", J )
        dq = lam * np.matmul(np.linalg.inv(J), err);
        print("dq ",dq)
        #print("idx ", idx)
        MoveJoints(idx, dq);
        ForwardKinematics(1);
        err = CalcVWerr(Target, arr[to]);
        #print("err ", err)

    err_norm = np.linalg.norm(err);
    return err_norm

In [ ]:
def MoveJoints(idx, dq):
    for n in range(len(idx)):
        j = idx[n];
        arr0 = arr[j].q
        arr[j].q = arr[j].q + dq[n];
        #print("arr1, arr0, dq ",arr[n].q , arr0 , dq[n])
    #print("arr_q ", arr[0].q, arr[1].q, arr[2].q, arr[3].q, arr[4].q, arr[5].q)

In [ ]:
#ヤコビ行列を使った逆運動学計算
#初期姿勢がおかしいと計算できないことがあるので注意
Rfoot=uLINK("Rfoot_target",0,0,0,0,0,0,0,R0,p0)
Lfoot=uLINK("Lfoot_target",0,0,0,0,0,0,0,R0,p0)
RLEG_J5 = 7;
LLEG_J5 = 13;

Rfoot.p = [0.0, -0.1, -0.3]; # [0, -0.1, 0] + 0.2*(np.random.rand(3)-0.5);
Rfoot.R = np.eye(3); #RPY2R(1/2*math.pi*(np.random.rand(3)-0.5));#   -pi/4 < q < pi/4
rerr_norm = InverseKinematics(RLEG_J5, Rfoot);

Lfoot.p = [0, 0.1, -0.4]; # [0, 0.1, 0] + 0.2*(np.random.rand(3)-0.5);
Lfoot.R = np.eye(3); #RPY2R(1/2*math.pi*(np.random.rand(3)-0.5));#   -pi/4 < q < pi/4
lerr_norm = InverseKinematics(LLEG_J5, Lfoot);

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
r_leg = np.array([arr[1].p,arr[2].p,arr[3].p,arr[4].p,arr[5].p,arr[6].p,arr[7].p])
l_leg = np.array([arr[1].p,arr[8].p,arr[9].p,arr[10].p,arr[11].p,arr[12].p,arr[13].p])

ax.plot(r_leg[:,0],r_leg[:,1],r_leg[:,2],"o-", c='b')
ax.plot(l_leg[:,0],l_leg[:,1],l_leg[:,2],"o-", c='b')
ax.set_xticks(np.linspace(-1.0, 1.0, 5))
ax.set_yticks(np.linspace(-1.0, 1.0, 5))
ax.set_zticks(np.linspace(-1.0, 1.0, 5))
#inline
plt.show()
print("arr[7].p ", arr[7].p)
print("rerr_norm ", rerr_norm)
print("arr[13].p ", arr[13].p)
print("lerr_norm ", lerr_norm)

In [ ]:
def Draw_R(arr):
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
    r_leg = np.array([arr[1].p,arr[2].p,arr[3].p,arr[4].p,arr[5].p,arr[6].p,arr[7].p])
    l_leg = np.array([arr[1].p,arr[8].p,arr[9].p,arr[10].p,arr[11].p,arr[12].p,arr[13].p])

    ax.plot(r_leg[:,0],r_leg[:,1],r_leg[:,2],"o-", c='b')
    ax.plot(l_leg[:,0],l_leg[:,1],l_leg[:,2],"o-", c='b')
    ax.set_xticks(np.linspace(-1.0, 1.0, 5))
    ax.set_yticks(np.linspace(-1.0, 1.0, 5))
    ax.set_zticks(np.linspace(-1.0, 1.0, 5))
    #inline
    plt.show()

In [ ]:
Draw_R(arr)

## ４．逆運動学２

LM法を使ったロバストな逆運動学の計算．

In [ ]:
def SetJointAngles(idx, q):
    for n in range(len(idx)):
        j = idx[n];
        arr[j].q = q[n];
        
    ForwardKinematics(1);

In [ ]:

def InverseKinematics_LM(to, Target):
# Levenberg-Marquardt, Chan-Lawrence, Sugihara's modification

    idx = FindRoute(to);
    wn_pos = 1/0.3;
    wn_ang = 1/(2*math.pi);
    We = np.diag([wn_pos, wn_pos, wn_pos, wn_ang, wn_ang, wn_ang]);#何やってる？
    Wn = np.eye(len(idx));#何やってるの？

    ForwardKinematics(1);
    err = CalcVWerr(Target, arr[to]);
    Ek = np.matmul(np.matmul(err, We),err); #Ek = err'*We*err;

    for n in range(10):
        J  = CalcJacobian(idx);
        #Jh = J'*We*J + Wn*(Ek + 0.002);  Hk + wn
        Jh = np.matmul(np.matmul(J.T,We),J) + Wn*(Ek+0.002)
        #gerr = J'*We*err;    gk
        gerr = np.matmul(np.matmul(J.T, We), err);
        #dq   = Jh \ gerr;    new
        dq = np.matmul(np.linalg.inv(Jh), gerr);
            
        MoveJoints(idx, dq);
        ForwardKinematics(1);
        err = CalcVWerr(Target, arr[to]);
        Ek2 = np.matmul(np.matmul(err, We),err);
        #print("Ek2 ",Ek2)
        #Ek2 = err'*We*err;
        if Ek2 < 1e-12:
          break;
        elif Ek2 < Ek:
          Ek = Ek2;
        else:
          MoveJoints(idx, -dq);#   revert
          ForwardKinematics(1);
        break; 
        #end
    #end

    #if nargout == 1 
    err_norm = np.linalg.norm(err);
    #end

    return err_norm

In [ ]:
# Numerical inverse kinematics and knee stretch
# ik_stretch_LM.m
# 2013 Sept.23 s.kajita AIST

SkipFrame = 10;#描画のスキップ

RLEG_J0 = 2;
RLEG_J5 = 7;
LLEG_J5 = 13;

# target foot position and orientation 
idx   = FindRoute(RLEG_J5);

target = np.array([0.0, 0.0, -25.0, 50.0, -25.0, 0.0])#deg
SetJointAngles(idx, target*ToRad);

Rfoot=uLINK("Rfoot_target",0,0,0,0,0,0,0,R0,p0)
#Lfoot=uLINK("Lfoot_target",0,0,0,0,0,0,0,R0,p0)
Rfoot.p = [0.0, -0.1, -0.4]; # [0, -0.1, 0] + 0.2*(np.random.rand(3)-0.5);
Rfoot.R = np.eye(3); #RPY2R(1/2*math.pi*(np.random.rand(3)-0.5));#    -pi/4 < q < pi/4

#Lfoot.p = [0, 0.1, -0.4]; # [0, 0.1, 0] + 0.2*(np.random.rand(3)-0.5);
#Lfoot.R = np.eye(3); #RPY2R(1/2*math.pi*(np.random.rand(3)-0.5));#    -pi/4 < q < pi/4


Height = np.linalg.norm(arr[RLEG_J0].p - arr[RLEG_J5].p);
LegLength = 0.6;
SingularPoint = math.sqrt(0.6*0.6 - Height*Height);

#ここからまだ修正していない
#xd_m = 0:0.005:0.4';
Nstep = 150 #length(xd_m);
#q_m = zeros(Nstep,6);
#analy_q_m = zeros(Nstep,6);
#x_m = zeros(Nstep,1);
#manip_m = zeros(Nstep,1);

print("Rfoot ", Rfoot.p[0] );

for n in range(Nstep):
    Rfoot.p[0] = 0.0 + 0.005 * n;  #右足のxを更新　 Right foot

    #print("Rfoot ", Rfoot.p[0] );
    #print("arr[RLEG_J5].p[0]",arr[RLEG_J5].p[0])

    rerr_norm = InverseKinematics_LM(RLEG_J5, Rfoot);
    
    #print("arr_p[0] ", arr[RLEG_J5].p[0]);
    #q_m(n,:) = [uLINK(idx).q];
    #analy_q_m(n,:) = IK_leg(uLINK(BODY),-0.1,0.3,0.3,Rfoot)';     # Analytical inverse kinematics solution

    #J = CalcJacobian(idx);
    #manip_m(n) = abs(det(J));
    
    if np.mod(n,SkipFrame) == 1:
        #print("arr[RLEG_J5].p[0]",arr[RLEG_J5].p[0])
        Draw_R(arr);
        print("Error ", rerr_norm);





In [ ]:
# ik_strechのgifアニメーションセーブバージョン

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

SkipFrame = 10;#描画のスキップ

RLEG_J0 = 2;
RLEG_J5 = 7;
LLEG_J5 = 13;

# target foot position and orientation
idx   = FindRoute(RLEG_J5);

target = np.array([0.0, 0.0, -25.0, 50.0, -25.0, 0.0])#deg
SetJointAngles(idx, target*ToRad);

Rfoot=uLINK("Rfoot_target",0,0,0,0,0,0,0,R0,p0)
#Lfoot=uLINK("Lfoot_target",0,0,0,0,0,0,0,R0,p0)
Rfoot.p = [0.0, -0.1, -0.4]; # [0, -0.1, 0] + 0.2*(np.random.rand(3)-0.5);
Rfoot.R = np.eye(3); #RPY2R(1/2*math.pi*(np.random.rand(3)-0.5));#    -pi/4 < q < pi/4

#Lfoot.p = [0, 0.1, -0.4]; # [0, 0.1, 0] + 0.2*(np.random.rand(3)-0.5);
#Lfoot.R = np.eye(3); #RPY2R(1/2*math.pi*(np.random.rand(3)-0.5));#    -pi/4 < q < pi/4


Height = np.linalg.norm(arr[RLEG_J0].p - arr[RLEG_J5].p);
LegLength = 0.6;
SingularPoint = math.sqrt(0.6*0.6 - Height*Height);


#xd_m = 0:0.005:0.4';
Nstep = 100 #length(xd_m);
#q_m = zeros(Nstep,6);
#analy_q_m = zeros(Nstep,6);
#x_m = zeros(Nstep,1);
#manip_m = zeros(Nstep,1);

print("Rfoot ", Rfoot.p[0] );

data = np.array([arr[1].p,arr[2].p,arr[3].p,arr[4].p,arr[5].p,arr[6].p,arr[7].p])

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})

ims = []

def update(num, data, line):
    line.set_data(data[:2, :num])
    line.set_3d_properties(data[2, :num])


for n in range(Nstep):
    Rfoot.p[0] = 0.0 + 0.005 * n;  #右足のxを更新　 Right foot

    #print("Rfoot ", Rfoot.p[0] );
    #print("arr[RLEG_J5].p[0]",arr[RLEG_J5].p[0])

    rerr_norm = InverseKinematics_LM(RLEG_J5, Rfoot);

    r_leg = np.array([arr[1].p,arr[2].p,arr[3].p,arr[4].p,arr[5].p,arr[6].p,arr[7].p])
    l_leg = np.array([arr[1].p,arr[8].p,arr[9].p,arr[10].p,arr[11].p,arr[12].p,arr[13].p])
    data = np.vstack((data,r_leg));
    
    #print("arr_p[0] ", arr[RLEG_J5].p[0]);
    #q_m(n,:) = [uLINK(idx).q];
    #analy_q_m(n,:) = IK_leg(uLINK(BODY),-0.1,0.3,0.3,Rfoot)';      # Analytical inverse kinematics solution

    #J = CalcJacobian(idx);
    #manip_m(n) = abs(det(J));
    
    if np.mod(n,SkipFrame) == 1:
        #print("arr[RLEG_J5].p[0]",arr[RLEG_J5].p[0])
        #Draw_R(arr);
        #Gen_R(arr);
        r_leg = np.array([arr[1].p,arr[2].p,arr[3].p,arr[4].p,arr[5].p,arr[6].p,arr[7].p])
        l_leg = np.array([arr[1].p,arr[8].p,arr[9].p,arr[10].p,arr[11].p,arr[12].p,arr[13].p])
        im1, = ax.plot(r_leg[:,0],r_leg[:,1],r_leg[:,2],"o-", c='b')
        im2, = ax.plot(l_leg[:,0],l_leg[:,1],l_leg[:,2],"o-", c='b')
        ims.append([im1,im2])
        print("Error ", rerr_norm);
        
ax.set_xticks(np.linspace(-1.0, 1.0, 5))
ax.set_yticks(np.linspace(-1.0, 1.0, 5))
ax.set_zticks(np.linspace(-1.0, 1.0, 5))
ani = animation.ArtistAnimation(fig, ims, interval=100)
ani.save('anim.gif', writer="pillow")
#ani.save('anim.mp4', writer="ffmpeg") # 動画はこちらをコメントアウトを外せば作成できる
plt.show()

## ５．動力学

動力学の計算を行う．リンク単体の動きから確認していく．

In [ ]:
import numpy as np
import math
import sys
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
arr=[0]*10

In [ ]:
#For Dynamic simulation dvo, w, vo　追加
class uLINK:
  def __init__(self, name, sister, child, mother, m,  b ,a ,q,R,p, dq, ddq, c, I, Ir, gr, u, w, vo, dw, dvo):
    self.name = name
    self.sister = sister
    self.child = child
    self.mother = mother
    self.m = m
    self.b = b
    self.a = a
    self.q = q
    self.R = R
    self.p = p
    self.dq =dq
    self.ddq = ddq
    self.c  = c
    self.I  = I
    self.u  = u
    self.w = w 
    self.vo = vo 
    self.dw = dw 
    self.dvo = dvo

In [ ]:
def MakeRigidBody(j, wdh, mass):
    arr[j].m = mass;
    arr[j].c = [0, 0, 0];
    arr[j].I = np.array([[1/12*(wdh[1]*wdh[1] + wdh[2]*wdh[2]), 0.0, 0.0],
            [0.0, 1/12*(wdh[0]*wdh[0] + wdh[2]*wdh[2]),  0.0],
            [0.0, 0.0, 1/12*(wdh[0]*wdh[0] + wdh[1]*wdh[1])]]) * mass;
    arr[j].vertex = np.array([
        [0.0,      0.0,      0.0],
        [0.0,      wdh[1], 0.0],
        [wdh[0],   wdh[1],  0.0],
        [wdh[0],   0.0,     0.0],
        [0.0,      0.0,     wdh[2]],
        [0.0,      wdh[1],  wdh[2]],
        [wdh[0],   wdh[1],  wdh[2]],
        [wdh[0],   0.0,     wdh[2]]
        ])
    for n in range(3):
        for i in range(8):
            arr[1].vertex[i,n] = arr[1].vertex[i,n] - wdh[n]/2;
        
    arr[j].face = [
        [1, 2, 3, 4], [2, 6, 7, 3], [4, 3, 7, 8],
        [1, 5, 8, 4], [1, 2, 6, 5], [5, 6, 7, 8]
        ];

In [ ]:
def EulerDynamics(j):
    I = np.matmul(np.matmul(arr[j].R, arr[j].I), arr[j].R.T); #     慣性テンソル
    L = np.matmul(I, arr[j].w);   # 角運動量
    arr[j].dw  = -np.matmul(np.linalg.inv(I), np.cross(arr[j].w, L)); #    Eulerの方程式
    return L

In [ ]:
def ShowObject(arr):
    vert = np.matmul(arr[1].R, arr[1].vertex.T);
    for k in range(3) :# 1:3
        vert[k,:] = vert[k,:] + arr[1].p[k];#    adding x,y,z to all vertex

    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
    im=ax.plot(vert[0,:],vert[1,:],vert[2,:],"o-", c='b')
    #ax.set_xticks(np.linspace(AX[0], AX[1], 5))
    #ax.set_yticks(np.linspace(AY[0], AY[1], 5))
    #ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))
    ax.set_xticks(np.linspace(AX[0]+arr[BODY].p[0], AX[1]+arr[BODY].p[0], 5))
    ax.set_yticks(np.linspace(AY[0]+arr[BODY].p[1], AY[1]+arr[BODY].p[1], 5))
    ax.set_zticks(np.linspace(AZ[0]+arr[BODY].p[2], AZ[1]+arr[BODY].p[2], 5))
    #ims.append(im);
    plt.show()


In [ ]:
#rigidbody_rotate.m
lx  = 0.1; ly  = 0.4; lz  = 0.9;
mass = 36.0; 
R0=np.eye(3);
p0= [0,0,0];

arr = [0]*2
arr[1]=uLINK("Rfoot_target",0,0,0,0,0,0,0,R0,p0,0,0,0,0,0,0,0,p0,p0,p0,p0)
boxsize=np.array([lx, ly, lz]);
MakeRigidBody(1, boxsize, mass); #直方体のデータを作成

arr[1].p = [0.0, 0.0, 0.0];
arr[1].R = np.eye(3); 
arr[1].w = [1, 1, 1];      # 初期角速度 [rad/s]
Dtime   = 0.001; #0.001s(1ms)にしないと本と同じような結果にはならない．サンプルプログラムは0.02s(20ms)だとかなり誤差が発生する．
EndTime = 10.0;

time  = int(EndTime//Dtime)+1;

L_w = np.zeros([time,3])
w_w = np.zeros([time,3])

#figure
AX=[-0.5, 0.5]; AY=[-0.5, 0.5]; AZ=[-0.5, 1.0];

for n in range(time):
    L_w[n,:] = EulerDynamics(1);
    arr[1].R = np.matmul(Rodrigues(arr[1].w, Dtime), arr[1].R);
    arr[1].w = arr[1].w + Dtime * arr[1].dw; 
    w_w[n,:] = arr[1].w;
    #print(arr[1].R)
    #ShowObject(arr);  # 姿勢を描画．

times = np.arange(0,EndTime,Dtime)
fig = plt.figure();
ax = fig.add_subplot(2, 1, 1) # 角運動量保存を確認するための図(サンプリングタイムを1msぐらいにしないと精度は悪い)
ax.plot(times, L_w[:,0])
ax.plot(times, L_w[:,1])
ax.plot(times, L_w[:,2])
ax = fig.add_subplot(2, 1, 2) # 各軸の角速度の変化
ax.plot(times, w_w[:,0])
ax.plot(times, w_w[:,1])
ax.plot(times, w_w[:,2])
plt.show()

In [ ]:
#rigidbody_rotate.m アニメーション出力
#matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

ims =[]
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
ax.set_xticks(np.linspace(AX[0], AX[1], 5))
ax.set_yticks(np.linspace(AY[0], AY[1], 5))
ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))

lx  = 0.1; ly  = 0.4; lz  = 0.9;
mass = 36.0; 
R0=np.eye(3);
p0= [0,0,0];

arr = [0]*2
arr[1]=uLINK("Rfoot_target",0,0,0,0,0,0,0,R0,p0,0,0,0,0,0,0,0,p0,p0,p0,p0)

MakeRigidBody(1, [lx, ly, lz], mass); #直方体のデータを作成

arr[1].p = [0.0, 0.0, 0.0];
arr[1].R = np.eye(3); 
arr[1].w = [1, 1, 1];      # 初期角速度 [rad/s]
Dtime   = 0.02;            # 20ms刻みなので，角運動量が一定にならない誤差が発生している
EndTime = 5.0;

time  = int(EndTime//Dtime)+1;
print(time)

#figure
AX=[-0.5, 0.5]; AY=[-0.5, 0.5]; AZ=[-0.5, 1.0];
L_w = np.zeros([time,3])
w_w = np.zeros([time,3])

for n in range(time):
    L_w[n,:] = EulerDynamics(1);
    arr[1].R = np.matmul(Rodrigues(arr[1].w, Dtime), arr[1].R);
    arr[1].w = arr[1].w + Dtime * arr[1].dw; 
    w_w[n,:] = arr[1].w;
    #print(arr[1].R)
    
    vert = np.matmul(arr[1].R, arr[1].vertex.T);
    for k in range(3) :# 1:3
        vert[k,:] = vert[k,:] + arr[1].p[k];#   adding x,y,z to all vertex


    im = ax.plot(vert[0,:],vert[1,:],vert[2,:],"o-", c='b')
    ims.append(im)
    

ani = animation.ArtistAnimation(fig, ims, interval=100)
ani.save('rotate.gif', writer="pillow")
plt.show()

times = np.arange(0,EndTime,Dtime)
fig = plt.figure();
ax = fig.add_subplot(2, 1, 1)
ax.plot(times, L_w[:,0])
ax.plot(times, L_w[:,1])
ax.plot(times, L_w[:,2])
ax = fig.add_subplot(2, 1, 2)
ax.plot(times, w_w[:,0])
ax.plot(times, w_w[:,1])
ax.plot(times, w_w[:,2])
plt.show()
    

In [ ]:
start = 50
stop = 100
step = 2
np.arange(stop)
#0 ≦ n < stopで間隔は1
np.arange(start, stop)
#start ≦ n < stopで間隔は1
np.arange(start, stop, step)
#start ≦ n < stopで間隔はstep

## ６．順運動学

単位ベクトル法による順運動学の計算．単一剛体の並進と回転の運動をシミュレーションするプログラム．

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import ArtistAnimation
fig = plt.figure(facecolor="w")
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)
# 0 <=x < 2pi の範囲の点列を作成。
x = np.linspace(0, 2*np.pi, 101)[: -1]
# 各コマの画像を格納する配列
image_list = []
# 1周だと短すぎたので5回繰り返す
for _ in range(1):
    for i in range(100):
        # ずらしながらsinカーブを描写し、配列に格納
        y1 = np.sin(np.roll(x, -i))
        y2 = np.cos(np.roll(x, -i))
        # 一つ目のグラフを描写する
        image1 = ax1.plot(x, y1, c="b")
        # 二つ目のグラフを描写する
        image2 = ax2.plot(x, y2, c="g")
        # 同時に描写したいグラフを連結したものを配列に追加する。
        image_list.append(image1+image2)
# アニメーションを作成
ani = ArtistAnimation(fig, image_list, interval=10)
# mp4ファイルに保存
ani.save('animation.mp4', writer='ffmpeg')
# gifファイルに保存する場合
# ani.save('animation.gif', writer='pillow')

In [ ]:
def DrawAllJoints(j):
    radius    = 0.02;
    len       = 0.06;
    joint_col = 0;

    if j != 0 : 
        #if arr[j].vertex.any() : #vertexが空じゃなかったら．~isempty(uLINK(j).vertex)
        if hasattr(arr[j], 'vertex') : #vertexメンバ変数を持っているかのチェック
            vert = np.matmul(arr[j].vertex, arr[j].R);
            for k in range(3): # 1:3
                vert[k,:] = vert[k,:] + arr[j].p[k]; # adding x,y,z to all vertex
            #ShowObject(arr) #DrawPolygon(vert, uLINK(j).face,0);
            vert = np.matmul(arr[1].R, arr[1].vertex.T);
            for k in range(3) :# 1:3
                vert[k,:] = vert[k,:] + arr[1].p[k];#    adding x,y,z to all vertex

            fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
            ax.plot(vert[0,:],vert[1,:],vert[2,:],"o-", c='b')
            #ax.set_xticks(np.linspace(AX[0], AX[1], 5))
            #ax.set_yticks(np.linspace(AY[0], AY[1], 5))
            #ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))
            ax.set_xticks(np.linspace(AX[0], AX[1], 5))
            ax.set_yticks(np.linspace(AY[0], AY[1], 5))
            ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))
            #ims.append(im);
            #plt.show()
    
        i = arr[j].mother;
        if i != 0:
            #Connect3D(uLINK(i).p,uLINK(j).p,'k',2);
            #function Connect3D(p1,p2,option,pt)
            #h = plot3([p1(1) p2(1)],[p1(2) p2(2)],[p1(3) p2(3)],option);
            #fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
            ax.plot([arr[i].p[0],arr[j].p[0]],[arr[i].p[1],arr[j].p[1]],[arr[i].p[2],arr[j].p[2]],"o-", c='b')
            #set(h,'LineWidth',pt)
            #ims.append(im)

        #DrawCylinder(uLINK(j).p, uLINK(j).R * uLINK(j).a, radius,len, joint_col);
    
    
        DrawAllJoints(arr[j].child);
        DrawAllJoints(arr[j].sister);


In [ ]:
def DrawAllJoints_org(j): #　DrawAllJointsバックアップ
    radius    = 0.02;
    len       = 0.06;
    joint_col = 0;

    if j != 0 : 
        #if arr[j].vertex.any() : #vertexが空じゃなかったら．~isempty(uLINK(j).vertex)
        if hasattr(arr[j], 'vertex') : #vertexメンバ変数を持っているかのチェック
            vert = np.matmul(arr[j].vertex, arr[j].R);
            for k in range(3): # 1:3
                vert[k,:] = vert[k,:] + arr[j].p[k]; # adding x,y,z to all vertex
            ShowObject(arr) #DrawPolygon(vert, uLINK(j).face,0);
    
        i = arr[j].mother;
        if i != 0:
            #Connect3D(uLINK(i).p,uLINK(j).p,'k',2);
            #function Connect3D(p1,p2,option,pt)
            #h = plot3([p1(1) p2(1)],[p1(2) p2(2)],[p1(3) p2(3)],option);
            #fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
            im=ax.plot([arr[i].p[0],arr[j].p[0]],[arr[i].p[1],arr[j].p[1]],[arr[i].p[2],arr[j].p[2]],"o-", c='b')
            #set(h,'LineWidth',pt)
            ims.append(im)

        #DrawCylinder(uLINK(j).p, uLINK(j).R * uLINK(j).a, radius,len, joint_col);
    
    
        DrawAllJoints(arr[j].child);
        DrawAllJoints(arr[j].sister);


In [ ]:
def hat(c):
    c_hat = np.array([[0, -c[2], c[1]], [c[2], 0., -c[0]], [-c[1], c[0], 0.]]);
    return c_hat
    

In [ ]:
# 縦ベクトル（3x1)掛ける横ベクトル(1x3)のサンプル：結果は３ｘ３の行列：演算子は「@」で計算できる．
# aの形状 (3, 1)：３行１列
a = np.array([[1],
              [2],
              [3]])
print(a.reshape([1,3]))
# bの形状 (1, 3)：１行３列
b = np.array([[1, 2, 3]])

c = a @ b
print(c)


In [ ]:
def SE3exp(j, dt):
# see Murray, Li, Sastry p.42
    norm_w = np.linalg.norm(arr[j].w);
    if norm_w < sys.float_info.epsilon :
        p2 = arr[j].p + arr[j].vo * dt;
        R2 = arr[j].R;
    else :
        th = norm_w*dt;
        wn = arr[j].w/norm_w;  # normarized vector
        vo = arr[j].vo/norm_w;
    
        w_wedge = np.array([[0., -wn[2], wn[1]],[wn[2], 0., -wn[0]],[-wn[1], wn[0], 0.]]);
        drot = w_wedge * math.sin(th) + np.matmul(w_wedge,w_wedge) * (1.0-math.cos(th));
        rot  = np.eye(3) + drot;
        #print("test2 ", wn.reshape([3,1])@wn.reshape([1,3])) # 「＠」は縦ベクトル(3x1)x横ベクトル(1x3)=３ｘ３行列の計算．あと，reshapeも必要．転置では対応できない
        #p2 = np.matmul(rot, arr[j].p) - np.matmul(drot, np.cross(wn,bb vo)) + np.dot(wn, wn.T)* vo * th;
        p2 = np.matmul(rot, arr[j].p) - np.matmul(drot, np.matmul(w_wedge, vo)) + np.matmul(wn.reshape([3,1])@wn.reshape([1,3]), vo) * th;
        R2 = np.matmul(rot, arr[j].R);
        
    return p2, R2

In [ ]:
# screw motionのシミュレーション．並進＋X軸回りの回転．図を逐次書き出し．
# #inline
import numpy as np #行列計算のためnumpyをインポート
import math        #三角関数を使うためにmathをインポート
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

#global uLINK G
ToRad = math.pi/180;
G = 9.8;
BODY=1;
WAIST = 1;
R0=np.eye(3);
p0= [0,0,0];
mass = 10.0; 
AX=[-1.0, 3.]; AY=[-2., 2.]; AZ=[-2., 2.0];
ims=[];

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
ax.set_xticks(np.linspace(AX[0], AX[1], 5))
ax.set_yticks(np.linspace(AY[0], AY[1], 5))
ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))

arr = [0]*2
arr[0]=uLINK("empty",0,0,0,0,0,0,0,R0,p0,0,0,0,0,0,0,0,p0,p0,p0,p0) # 空のゼロリンクを設定（ワールド座標系）
arr[0].vertex=np.array([]);
# def __init__(self, name, sister, child, mother, m, b ,a ,q,R,p, dq, ddq, c, I, Ir, gr, u, dw):
arr[BODY]=uLINK("BODY",0,0,0,mass,[0, 0, 0],[0.0, 0.0, 1.0],0,R0,p0,0,0,0,0,0,0,0,p0,p0,p0,p0)

#uLINK = struct('name','BODY'    , 'm', 10, 'sister', 0, 'child', 0, 'b',[0  0  0]','a',[0 0 1]','q',0);
#uLINK.mother = 0;

# Inertia tensor
lx  = 0.1;
ly  = 0.4;
lz  = 0.9;

#[uLINK(1).vertex,uLINK(1).face]   = MakeBox([lx ly lz]  ,[lx/2 ly/2 lz/2] );# BODY
wdh=[lx, ly, lz];
arr[BODY].vertex = np.array([
        [0.0,      0.0,      0.0],
        [0.0,      wdh[1], 0.0],
        [wdh[0],   wdh[1],  0.0],
        [wdh[0],   0.0,     0.0],
        [0.0,      0.0,     wdh[2]],
        [0.0,      wdh[1],  wdh[2]],
        [wdh[0],   wdh[1],  wdh[2]],
        [wdh[0],   0.0,     wdh[2]]
        ])
#print("arr.vertex1 ", arr[1].vertex);
for n in range(3): #原点移動
    for j in range(8):
        arr[BODY].vertex[j,n] = arr[BODY].vertex[j,n] - wdh[n]/2;
#print("arr.vertex2 ", arr[1].vertex);



# body state
#arr[WAIST]=uLINK("WAIST",0,0,mass,0,[0, 0, 0],[0.0, 0.0, 1.0],0,R0,p0,0,0,0,0,0,0,0,0)
#uLINK(WAIST).p = [0, 0, 0]';
#uLINK(WAIST).R = eye(3);
arr[WAIST].w = [1, 0, 0]; #uLINK(WAIST).w  = [1, 0, 0]';
arr[WAIST].vo = [0.3, 0, 0]; # uLINK(WAIST).vo = [0.3 0 1]';
#arr[WAIST].vertex=[];

for n in range(len(arr)):
    arr[n].u = 0.0;

Dtime = 0.3;
time  = 10.0;
tsize = int(time//Dtime); # length(time);「//」は商だけ求める演算子
print("tsize ",tsize)
for n in range(tsize):
    DrawAllJoints(1);
    plt.show();
    [arr[WAIST].p, arr[WAIST].R] = SE3exp(1, Dtime);       


In [ ]:
# screw のシミュレーション．並進移動vo = [0.3, 0, 1.0]＋x軸回りの回転w  = [1, 0, 0]
# アニメーション作成（screwmotion.gif）
#inline
import numpy as np #行列計算のためnumpyをインポート
import math        #三角関数を使うためにmathをインポート
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

#global uLINK G
ToRad = math.pi/180;
G = 9.8;
BODY=1;
WAIST = 1;
R0=np.eye(3);
p0= [0,0,0];
mass = 10.0; 
AX=[-1.0, 3.]; AY=[-2., 2.]; AZ=[-2., 2.0];
ims=[];

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
ax.set_xticks(np.linspace(AX[0], AX[1], 5))
ax.set_yticks(np.linspace(AY[0], AY[1], 5))
ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))

arr = [0]*2
arr[0]=uLINK("empty",0,0,0,0,0,0,0,R0,p0,0,0,0,0,0,0,0,p0,p0,p0,p0) # 空のゼロリンクを設定（ワールド座標系）
arr[0].vertex=np.array([]);
# def __init__(self, name, sister, child, mother, m, b ,a ,q,R,p, dq, ddq, c, I, Ir, gr, u, dw):
arr[BODY]=uLINK("BODY",0,0,0,mass,[0, 0, 0],[0.0, 0.0, 1.0],0,R0,p0,0,0,0,0,0,0,0,p0,p0,p0,p0)

#uLINK = struct('name','BODY'    , 'm', 10, 'sister', 0, 'child', 0, 'b',[0  0  0]','a',[0 0 1]','q',0);
#uLINK.mother = 0;

# Inertia tensor
lx  = 0.1;
ly  = 0.4;
lz  = 0.9;

wdh=[lx, ly, lz];
arr[BODY].vertex = np.array([
        [0.0,      0.0,      0.0],
        [0.0,      wdh[1], 0.0],
        [wdh[0],   wdh[1],  0.0],
        [wdh[0],   0.0,     0.0],
        [0.0,      0.0,     wdh[2]],
        [0.0,      wdh[1],  wdh[2]],
        [wdh[0],   wdh[1],  wdh[2]],
        [wdh[0],   0.0,     wdh[2]]
        ])

for n in range(3): #原点移動
    for j in range(8):
        arr[BODY].vertex[j,n] = arr[BODY].vertex[j,n] - wdh[n]/2;
#print("arr.vertex2 ", arr[1].vertex);



# body state
#arr[WAIST]=uLINK("WAIST",0,0,mass,0,[0, 0, 0],[0.0, 0.0, 1.0],0,R0,p0,0,0,0,0,0,0,0,0)
#uLINK(WAIST).p = [0, 0, 0]';
#uLINK(WAIST).R = eye(3);
arr[WAIST].w = [1, 0, 0]; #uLINK(WAIST).w  = [1, 0, 0]'; # 角速度の初速
arr[WAIST].vo = [0.3, 0, 1.0]; # uLINK(WAIST).vo = [0.3 0 1]'; # 並進速度の初速
#arr[WAIST].vertex=[];

for n in range(len(arr)):
    arr[n].u = 0.0;

Dtime = 0.3;
time  = 10.0;
tsize = int(time//Dtime); # length(time);「//」は商だけ求める演算子
print("tsize ",tsize)
for n in range(tsize):
    #DrawAllJoints(1);
    vert = np.matmul(arr[1].R, arr[1].vertex.T);
    for k in range(3): # 1:3
        vert[k,:] = vert[k,:] + arr[1].p[k]; # adding x,y,z to all vertex
    im = ax.plot(vert[0,:],vert[1,:],vert[2,:],"o-", c='b')
    ax.set_xticks(np.linspace(AX[0], AX[1], 5))
    ax.set_yticks(np.linspace(AY[0], AY[1], 5))
    ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))
    ims.append(im)
    [arr[WAIST].p, arr[WAIST].R] = SE3exp(1, Dtime);       

    
ani = animation.ArtistAnimation(fig, ims, interval=100)
ani.save('screwmotion.gif', writer="pillow")
plt.show()


In [ ]:
def hat(c):
    c_hat = np.array([[0, -c[2], c[1]], [c[2], 0., -c[0]], [-c[1], c[0], 0.]]);
    return c_hat
    

In [ ]:
def SE3dynamics(j,f,tau):

    w_c = np.matmul(arr[j].R, arr[j].c) + arr[j].p;   # center of mass
    w_I = np.matmul(np.matmul(arr[j].R, arr[j].I),  arr[j].R.T);  # inertia in world frame
    c_hat = hat(w_c);
    Iww = w_I + arr[j].m * np.matmul(c_hat,  c_hat.T); 
    Ivv = arr[j].m * np.eye(3);
    Iwv = arr[j].m * c_hat;

    #AB()で使うために保存 20220527 そもそもこの関数をどこでコールするべきなのか？
    arr[j].Iww = Iww;
    arr[j].Ivv = Ivv;
    arr[j].Ivw = Iwv;

    P = arr[j].m * (arr[j].vo + np.cross(arr[j].w, w_c));     #  linear  momentum
    L = arr[j].m * np.cross(w_c, arr[j].vo) + np.matmul(Iww, arr[j].w); #  angular momentum
    
    #pp = [];
    pp = np.hstack([np.cross(arr[j].w, P),
        np.cross(arr[j].vo,P) + np.cross(arr[j].w, L)]);
    pp = pp - np.hstack([f, tau]);   # 外力，外モーメントを加算
   
    Ia = np.vstack([np.hstack([Ivv, Iwv.T]), np.hstack([Iwv, Iww])]);
    #print("Iwv ", Iwv)
    #print("Ia ", Ia)
    a0 = -np.matmul(np.linalg.inv(Ia),  pp);
    #print("a0 ", a0)
    for k in range(3):
        arr[j].dvo[k] = a0[k];
        arr[j].dw[k] = a0[k+3];
      
    return P, L

In [ ]:
# rigidbody_fly 並進＋X,Z軸回りの回転
#inline
import numpy as np #行列計算のためnumpyをインポート
import sys         #計算機イプシロンを使うためにsysをインポート
import math        #三角関数を使うためにmathをインポート
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

#global uLINK G
ToRad = math.pi/180;
G = 9.8;
BODY=1;
R0=np.eye(3);
p0= np.zeros(3);
mass = 36.0; 
AX=[-1.0, 2.5]; AY=[-2., 2.]; AZ=[-2., 2.0];
ims=[];

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
#ax.set_xticks(np.linspace(AX[0], AX[1], 5))
#ax.set_yticks(np.linspace(AY[0], AY[1], 5))
#ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))

arr = [0]*2
arr[0]=uLINK("empty",0,0,0,0,0,0,0,np.eye(3),np.zeros(3),0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3)) # 空のゼロリンクを設定（ワールド座標系）
arr[0].vertex=np.array([]);
# def __init__(self, name, sister, child, m, mother, b ,a ,q,R,p, dq, ddq, c, I, Ir, gr, u, dw):
arr[BODY]=uLINK("BODY",0,0,0,mass,np.zeros(3),np.array([0.0, 0.0, 1.0]),0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
#uLINK = struct('name','BODY' , 'sister', 0, 'child', 0, 'b',[0  0  0]','a',[0 0 1]','mother',0);

# Inertia tensor
lx  = 0.1;
ly  = 0.4;
lz  = 0.9;

wdh=[lx, ly, lz];
arr[BODY].vertex = np.array([
        [0.0,      0.0,      0.0],
        [0.0,      wdh[1], 0.0],
        [wdh[0],   wdh[1],  0.0],
        [wdh[0],   0.0,     0.0],
        [0.0,      0.0,     wdh[2]],
        [0.0,      wdh[1],  wdh[2]],
        [wdh[0],   wdh[1],  wdh[2]],
        [wdh[0],   0.0,     wdh[2]]
        ])
for n in range(3): #原点移動
    for j in range(8):
        arr[BODY].vertex[j,n] = arr[BODY].vertex[j,n] - wdh[n]/2;


arr[BODY].I = np.array([[ 1/12*(ly*ly + lz*lz), 0., 0.],
            [0., 1/12*(lx*lx + lz*lz),  0],
            [0., 0., 1/12*(lx*lx + ly*ly)]]) * arr[BODY].m;
arr[BODY].c = np.array([0., 0., 0.]);
        
# initial state
arr[BODY].p = np.array([0.0, 0.0, 0.0]);
arr[BODY].R = np.eye(3);
arr[BODY].w = np.array([1., 0., 1.]); #回転ベクトル
arr[BODY].vo= np.array([0.5, 0.1, 0.]); #並進速度

Dtime   = 0.005;
EndTime = 5.0;
tsize  = int(EndTime//Dtime);
frame_skip = 60;

L_m = np.zeros((tsize,3));
P_m = np.zeros((tsize,3));
w_m = np.zeros((tsize,3));
vo_m = np.zeros((tsize,3));

for n in range(tsize):
    vo_m[n,:] = arr[BODY].vo;
    w_m[n,:] = arr[BODY].w;
    if n == 0:
        time = [0]
    else:
        time = np.hstack([time, n*Dtime])
    [P,L] = SE3dynamics(1, np.zeros(3), np.zeros(3));
    P_m[n,:] = P.T;
    L_m[n,:] = L.T;
    
    [arr[BODY].p, arr[BODY].R] = SE3exp(1, Dtime);
    arr[BODY].w = arr[BODY].w + Dtime * arr[BODY].dw;
    arr[BODY].vo= arr[BODY].vo+ Dtime * arr[BODY].dvo;
        
    if np.mod(n,frame_skip) == 1:
        #DrawAllJoints(1);
        vert = np.matmul(arr[1].R, arr[1].vertex.T);
        for k in range(3) :# 1:3
            vert[k,:] = vert[k,:] + arr[1].p[k];# adding x,y,z to all vertex

        #fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
        im=ax.plot(vert[0,:],vert[1,:],vert[2,:],"o-", c='b')
        ax.set_xticks(np.linspace(AX[0], AX[1], 5))
        ax.set_yticks(np.linspace(AY[0], AY[1], 5))
        ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))
        ims.append(im);

ani = animation.ArtistAnimation(fig, ims, interval=100)
ani.save('rigidbody_fly.gif', writer="pillow")
plt.show()

fig = plt.figure();
ax = fig.add_subplot(2, 1, 1)
ax.plot(time, vo_m[:,0])
ax.plot(time, vo_m[:,1])
ax.plot(time, vo_m[:,2])
ax = fig.add_subplot(2, 1, 2)
ax.plot(time, P_m[:,0])
ax.plot(time, P_m[:,1])
ax.plot(time, P_m[:,2])
plt.show()

### コマのシミュレーション


In [ ]:
def MakeZcylinder(pos, radius,len):

    a = 10;    # number of side faces

    for i in range(a):
        theta = i/a * 2*math.pi;
        if i == 0:
            x = np.array(radius*math.cos(theta))
            y = np.array(radius*math.sin(theta))
        x  = np.hstack([x, radius*math.cos(theta)]);
        y  = np.hstack([y, radius*math.sin(theta)]);

    z1 = len/2 * np.ones(a+1); #ones(1,a);
    z2 = -z1;
    #print("x ", x)
    #print("y ", y)
    #print("z1 ", z1)

    vert    = np.vstack([np.hstack([x, x, 0., 0.]),
            np.hstack([y, y, 0., 0.]),
            np.hstack([z1, z2, len/2., -len/2.])]);
    for n in range(3): # 1:3
        vert[n,:] = vert[n,:] + pos[n];
    
    #print("vert ", vert)
 
    # make index data specifying polygons
    #face_side = [1:a; a+1:2*a; a+2:2*a a+1; 2:a 1];
    #face_up   = [1:a; 2:a 1];
    #face_up(3:4,:) = 2*a+1;  # index of up center
    #face_down = [a+2:2*a a+1; a+1:2*a];
    #face_down(3:4,:) = 2*a+2; # index of down center
    #face = [face_side face_up face_down];

    return vert


In [ ]:
def MakeTop(j, r, a, c):
    pos = np.array([0., 0., c]);
    vertex1 = MakeZcylinder(pos, r, a);             # 円盤部の形状
    vertex2 = MakeZcylinder(pos, 0.01, 2*c); # 軸の形状
    vertex = np.hstack([vertex1, vertex2]);  # １つに合成
    arr[j].vertex = vertex.T;                # 転置しておいてShowObjectに渡せるようにしている．

    density = 2.7E+3;                     #  アルミニウムの比重　[kg/m^2]
    arr[j].m = math.pi*r*r*a*density;         # 円盤部の質量 [kg]
    arr[j].I = np.array([ [(a*a + 3*r*r)/12, 0., 0.],   
                [0., (a*a + 3*r*r)/12.,  0.],
                [0., 0., r*r/2]]) * arr[j].m;   # 円盤部の慣性テンソル
    arr[j].c = [0., 0., c];                 # 重心


In [ ]:
def TopForce(j):
    w_c = np.matmul(arr[j].R, arr[j].c) + arr[j].p;   # 重心位置
    f = np.array([0., 0., -arr[j].m * G]);  # 重力
    t = np.cross(w_c, f);                   # 重力による原点回りモーメント

    if arr[j].p[2] < 0.0 : # 支点は床面と接触している
        Kf = 1.0E+4;      # 床面の剛性[N/m]
        Df = 1.0E+3;      # 床面の粘性[N/(m/s)]
        v = arr[j].vo + np.cross(arr[j].w, arr[j].p);  # 支点の速度
        fc = np.array([-Df*v[0],  -Df*v[1], -Kf*arr[j].p[2]-Df*v[2]]);
        f = f + fc;
        t = t + np.cross(arr[j].p, fc);

    return f,t

In [ ]:
# top_simulation.m
# コマの運動のシミュレーション
#inline
import numpy as np #行列計算のためnumpyをインポート
import sys         #計算機イプシロンを使うためにsysをインポート
import math        #三角関数を使うためにmathをインポート
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})

R0=np.eye(3);
p0= [0,0,0];
BODY = 1;

arr = [0]*2
arr[0]=uLINK("empty",0,0,0,0,0,0,0,np.eye(3),np.zeros(3),0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3)) # 空のゼロリンクを設定（ワールド座標系）
arr[0].vertex=np.array([]);
# def __init__(self, name, sister, child, mother,　m,  b ,a ,q,R,p, dq, ddq, c, I, Ir, gr, u, dw):
arr[BODY]=uLINK("empty",0,0,0,0,0,0,0,np.eye(3),np.zeros(3),0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3)) 
#uLINK = struct('name','BODY' , 'sister', 0, 'child', 0, 'b',[0  0  0]','a',[0 0 1]','mother',0);

G = 9.8;
r = 0.2; a = 0.05; c = 0.2;#           # 円盤の半径,厚み，軸の長さ/2 [m]
MakeTop(1, r,a,c);


arr[1].p = np.array([0., 0., 0.3]);                # 初期位置 [m]
arr[1].R = Rodrigues([1., 0., 0.],math.pi/50.); # 初期姿勢
arr[1].vo= [0., 0., 0.];                 # 初速度 [m/s]
arr[1].w = [0., 0., 50.];                # 初期角速度 [rad/s]
Dtime   = 0.002;
EndTime = 2.0;
time  = int(EndTime//Dtime);
frame_skip = 3;

ims = [] # 空のイメージ保存用の配列用意

AX=[-0.2, 0.4];  AY=[-0.3, 0.3]; AZ=[0., 0.8];  # 3D表示範囲
for n in range(time):
    [f,tau] = TopForce(1);                           # 外力の計算
    [P,L]   = SE3dynamics(1,f,tau);                  # 加速度の計算    
    [arr[1].p, arr[1].R] = SE3exp(1, Dtime);     # 位置・姿勢の更新       
    arr[1].w = arr[1].w + Dtime * arr[1].dw;   # 角速度の更新
    arr[1].vo= arr[1].vo+ Dtime * arr[1].dvo;  # 速度の更新
    if np.mod(n,frame_skip) == 1 :
        #ShowObject(arr);                               # 表示
        vert = np.matmul(arr[1].R, arr[1].vertex.T);
        for k in range(3) :# 1:3
            vert[k,:] = vert[k,:] + arr[1].p[k];#   adding x,y,z to all vertex
        #fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'})
        im=ax.plot(vert[0,:],vert[1,:],vert[2,:],"o-", c='b')
        ax.set_xticks(np.linspace(AX[0], AX[1], 5))
        ax.set_yticks(np.linspace(AY[0], AY[1], 5))
        ax.set_zticks(np.linspace(AZ[0], AZ[1], 5))
        ims.append(im);

ani = animation.ArtistAnimation(fig, ims, interval=100)
ani.save('top_anim.gif', writer="pillow")
plt.show()



## ７．ヒューマノイドロボットの動力学シミュレーション
全ての関節トルクゼロで床に落ちていくシミュレーション．<br>
足裏だけ接触判定を行っている．

In [ ]:
def SetupRigidBody(j, wdh, com): #ほぼMakeRigidBodyと同じ．引数の違いは第３引数の質量と重心位置の違い
    #arr[j].m = mass;
    arr[j].c = com;
    arr[j].I = np.array([[1/12*(wdh[1]*wdh[1] + wdh[2]*wdh[2]), 0.0, 0.0],
            [0.0, 1/12*(wdh[0]*wdh[0] + wdh[2]*wdh[2]),  0.0],
            [0.0, 0.0, 1/12*(wdh[0]*wdh[0] + wdh[1]*wdh[1])]]) * arr[j].m;
    arr[j].vertex = np.array([
        [0.0,      0.0,      0.0],
        [0.0,      wdh[1], 0.0],
        [wdh[0],   wdh[1],  0.0],
        [wdh[0],   0.0,     0.0],
        [0.0,      0.0,     wdh[2]],
        [0.0,      wdh[1],  wdh[2]],
        [wdh[0],   wdh[1],  wdh[2]],
        [wdh[0],   0.0,     wdh[2]]
        ])
    for n in range(3):
        for i in range(8):
            arr[j].vertex[i,n] = arr[j].vertex[i,n] - wdh[n]/2 + com[n];

    arr[j].face = [
        [1, 2, 3, 4], [2, 6, 7, 3], [4, 3, 7, 8],
        [1, 5, 8, 4], [1, 2, 6, 5], [5, 6, 7, 8]
        ];

In [ ]:
# ２足歩行ロボットのモデルを動力学対応のため再定義
def SetupRobot(arr): # arr = [0]*14（１３リンク＋ゼロ番目の空のリンク）を引数にしないとだめ．
    ToDeg = 180/math.pi;   #r adianからdegreeに変換するときの係数
    ToRad = math.pi/180;   #d egreeからradianに変換するときの係数

    UX = [1.0,0.0,0.0];    # X軸の単位ベクトルで関節軸を設定するためのシンボル設定
    UY = [0.0,1.0,0.0];    # Y軸の単位ベクトルで関節軸を設定するためのシンボル設定
    UZ = [0.0,0.0,1.0];    # Z軸の単位ベクトルで関節軸を設定するためのシンボル設定

    R0=np.eye(3);          # 単位行列のシンボル設定
    p0= [0,0,0];           # ゼロ位置のシンボル設定

    #arr = [0]*14          # リンクの数（この例では14リンク）の配列を初期化

    # def __init__(self, name, sister, child, m, mother, b ,a ,q,R,p, dq, ddq, c, I, Ir, gr, u, dw):
    arr[0]=uLINK("empty",0,0,0,0,0,0,0,np.eye(3),np.zeros(3),0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3)) # 空のゼロリンクを設定（ワールド座標系）
    #arr[0].vertex=np.array([]);
    arr[1]=uLINK("BODY",0, 2, 0, 10, [0.0, 0.0, 0.7],UZ,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))

    # 右足のリンクの初期化：第3関節（股関節のピッチ）と第4関節（膝関節）のピッチ角を与えている）
    arr[2]=uLINK("RLEG_J0", 8, 3, 0, 5, [0.0,-0.1, 0.0],UZ,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[3]=uLINK("RLEG_J1", 0, 4, 0, 1, [0.0, 0.0, 0.0],UX,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[4]=uLINK("RLEG_J2", 0, 5, 0, 5, [0.0, 0.0, 0.0],UY,-90*ToRad,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[5]=uLINK("RLEG_J3", 0, 6, 0, 1, [0.0, 0.0,-0.3],UY,90*ToRad,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[6]=uLINK("RLEG_J4", 0, 7, 0, 6, [0.0, 0.0,-0.3],UY,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[7]=uLINK("RLEG_J5", 0, 0, 0, 2, [0.0, 0.0, 0.0],UX,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))

    # 左足のリンクの初期化：第10関節（股関節のピッチ）と第11関節（膝関節）のピッチ角を与えている）
    arr[8] =uLINK("LLEG_J0", 0,  9, 0, 5, [0.0, 0.1, 0.0],UZ,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[9] =uLINK("LLEG_J1", 0, 10, 0, 1, [0.0, 0.0, 0.0],UX,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[10]=uLINK("LLEG_J2", 0, 11, 0, 5, [0.0, 0.0, 0.0],UY,-90*ToRad,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[11]=uLINK("LLEG_J3", 0, 12, 0, 1, [0.0, 0.0,-0.3],UY,90*ToRad,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[12]=uLINK("LLEG_J4", 0, 13, 0, 6, [0.0, 0.0,-0.3],UY,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
    arr[13]=uLINK("LLEG_J5", 0,  0, 0, 2, [0.0, 0.0, 0.0],UX,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))

    FindMother(1)

    #フィールドの追加
    # for n=1:length(uLINK)
    #     uLINK(n).dq     = 0;             関節速度   [rad/s]
    #     uLINK(n).ddq    = 0;             関節加速度 [rad/s^2]
    #     uLINK(n).c      = [0 0 0]';      重心位置   [m]
    #     uLINK(n).I      = zeros(3,3);    重心回りの慣性テンソル [kg.m^2]
    #     uLINK(n).Ir     = 0.0;           モータの電気子慣性モーメント [kg.m^2]
    #     uLINK(n).gr     = 0.0;           モータの減速比 [-]
    #     uLINK(n).u      = 0.0;            関節トルク [Nm]
    # end
    for n in range(len(arr)):
        arr[n].c = np.zeros(3);
        arr[n].I = np.zeros((3,3));
        arr[n].Ir = 0.0;
        arr[n].gr = 0.0;


    #  プログラムを見やすくするため，リンクnameと同名の変数にID番号をセット
    for n in range(len(arr)):
        name_set = arr[n].name + "=" + str(n);
        exec(name_set);


    # 　胴体，および足部を剛体でモデル化
    shape = [0.1, 0.3, 0.5];     # 奥行き，幅，高さ [m]
    com   = [0, 0, 0.3];        # 重心位置
    SetupRigidBody(BODY, shape,com);

    shape = [0.2, 0.1, 0.02];    # 奥行き，幅，高さ [m]
    com   = [0.05,  0., -0.04];  # 重心位置
    SetupRigidBody(RLEG_J5, shape,com);

    shape = [0.2, 0.1, 0.02];     # 奥行き，幅，高さ [m]
    com   = [0.05,  0., -0.04];   # 重心位置
    SetupRigidBody(LLEG_J5, shape,com);

    # 特異姿勢にならないように足を曲げておく．各足のピッチ軸3つの初期角度を修正
    arr[RLEG_J2].q = -10.0*ToRad;
    arr[RLEG_J3].q = 20.0*ToRad;
    arr[RLEG_J4].q = -10.0*ToRad;

    arr[LLEG_J2].q = -10.0*ToRad;
    arr[LLEG_J3].q = 20.0*ToRad;
    arr[LLEG_J4].q = -10.0*ToRad;

    arr[BODY].p = [0.0, 0.0, 0.65]; # ボディの初期の高さを修正
    arr[BODY].R = np.eye(3);

    ForwardKinematics(1);

In [ ]:
#　２足歩行ロボットの初期化　SetupRobot(arr)を使わないときの宣言
ToDeg = 180/math.pi;   #r adianからdegreeに変換するときの係数
ToRad = math.pi/180;   #d egreeからradianに変換するときの係数

UX = [1.0,0.0,0.0];    # X軸の単位ベクトルで関節軸を設定するためのシンボル設定
UY = [0.0,1.0,0.0];    # Y軸の単位ベクトルで関節軸を設定するためのシンボル設定
UZ = [0.0,0.0,1.0];    # Z軸の単位ベクトルで関節軸を設定するためのシンボル設定

R0=np.eye(3);          # 単位行列のシンボル設定
p0= [0,0,0];           # ゼロ位置のシンボル設定

arr = [0]*14          # リンクの数（この例では14リンク）の配列を初期化

# def __init__(self, name, sister, child, m, mother, b ,a ,q,R,p, dq, ddq, c, I, Ir, gr, u, dw):
arr[0]=uLINK("empty",0,0,0,0,0,0,0,np.eye(3),np.zeros(3),0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3)) # 空のゼロリンクを設定（ワールド座標系）
#arr[0].vertex=np.array([]);
arr[1]=uLINK("BODY",0, 2, 0, 10, [0.0, 0.0, 0.7],UZ,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))

# 右足のリンクの初期化：第3関節（股関節のピッチ）と第4関節（膝関節）のピッチ角を与えている）
arr[2]=uLINK("RLEG_J0", 8, 3, 0, 5, [0.0,-0.1, 0.0],UZ,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[3]=uLINK("RLEG_J1", 0, 4, 0, 1, [0.0, 0.0, 0.0],UX,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[4]=uLINK("RLEG_J2", 0, 5, 0, 5, [0.0, 0.0, 0.0],UY,-90*ToRad,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[5]=uLINK("RLEG_J3", 0, 6, 0, 1, [0.0, 0.0,-0.3],UY,90*ToRad,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[6]=uLINK("RLEG_J4", 0, 7, 0, 6, [0.0, 0.0,-0.3],UY,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[7]=uLINK("RLEG_J5", 0, 0, 0, 2, [0.0, 0.0, 0.0],UX,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))

# 左足のリンクの初期化：第10関節（股関節のピッチ）と第11関節（膝関節）のピッチ角を与えている）
arr[8] =uLINK("LLEG_J0", 0,  9, 0, 5, [0.0, 0.1, 0.0],UZ,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[9] =uLINK("LLEG_J1", 0, 10, 0, 1, [0.0, 0.0, 0.0],UX,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[10]=uLINK("LLEG_J2", 0, 11, 0, 5, [0.0, 0.0, 0.0],UY,-90*ToRad,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[11]=uLINK("LLEG_J3", 0, 12, 0, 1, [0.0, 0.0,-0.3],UY,90*ToRad,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[12]=uLINK("LLEG_J4", 0, 13, 0, 6, [0.0, 0.0,-0.3],UY,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))
arr[13]=uLINK("LLEG_J5", 0,  0, 0, 2, [0.0, 0.0, 0.0],UX,0,R0,p0,0,0,0,0,0,0,0,np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3))

FindMother(1)

#フィールドの追加
# for n=1:length(uLINK)
#     uLINK(n).dq     = 0;             関節速度   [rad/s]
#     uLINK(n).ddq    = 0;             関節加速度 [rad/s^2]
#     uLINK(n).c      = [0 0 0]';      重心位置   [m]
#     uLINK(n).I      = zeros(3,3);    重心回りの慣性テンソル [kg.m^2]
#     uLINK(n).Ir     = 0.0;           モータの電気子慣性モーメント [kg.m^2]
#     uLINK(n).gr     = 0.0;           モータの減速比 [-]
#     uLINK(n).u      = 0.0;            関節トルク [Nm]
# end
for n in range(len(arr)):
    arr[n].c = np.zeros(3);
    arr[n].I = np.zeros((3,3));
    arr[n].Ir = 0.0;
    arr[n].gr = 0.0;


#  プログラムを見やすくするため，リンクnameと同名の変数にID番号をセット
for n in range(len(arr)):
    name_set = arr[n].name + "=" + str(n);
    exec(name_set);


# 　胴体，および足部を剛体でモデル化
shape = [0.1, 0.3, 0.5];     # 奥行き，幅，高さ [m]
com   = [0, 0, 0.3];        # 重心位置
SetupRigidBody(BODY, shape,com);

shape = [0.2, 0.1, 0.02];    # 奥行き，幅，高さ [m]
com   = [0.05,  0., -0.04];  # 重心位置
SetupRigidBody(RLEG_J5, shape,com);

shape = [0.2, 0.1, 0.02];     # 奥行き，幅，高さ [m]
com   = [0.05,  0., -0.04];   # 重心位置
SetupRigidBody(LLEG_J5, shape,com);

# 特異姿勢にならないように足を曲げておく．各足のピッチ軸3つの初期角度を修正
arr[RLEG_J2].q = -10.0*ToRad;
arr[RLEG_J3].q = 20.0*ToRad;
arr[RLEG_J4].q = -10.0*ToRad;

arr[LLEG_J2].q = -10.0*ToRad;
arr[LLEG_J3].q = 20.0*ToRad;
arr[LLEG_J4].q = -10.0*ToRad;

arr[BODY].p = [0.0, 0.0, 0.65]; # ボディの初期の高さを修正
arr[BODY].R = np.eye(3);

ForwardKinematics(1);

In [ ]:
#  プログラムを見やすくするため，リンクnameと同名の変数にID番号をセット
for n in range(len(arr)):
    name_set = arr[n].name + "=" + str(n);
    print(name_set)
    exec(name_set);
LLEG_J5

In [ ]:
# SetupRobot(arr)を使うときの宣言の方法
arr=[0]*14 # 今回は１3リンク分の２足歩行なので空のリンクも含めて長さ14の配列を渡す
SetupRobot(arr)
print(arr[BODY].p) # ボディの位置が初期化されたか確認
LLEG_J5 # リンクの名前にIDがセットされているか確認：LLEG_J5は13番

In [ ]:
#　重心位置計算のための再帰計算部分
def calcMC(j):
    if j == 0:
        mc = 0;
    else:
        mc = arr[j].m * (arr[j].p + np.matmul(arr[j].R, arr[j].c));
        mc = mc + calcMC(arr[j].sister) + calcMC(arr[j].child);
    
    return mc

In [ ]:
# 重心位置計算の関数
def calcCoM(): 
    M  = TotalMass(1); # 再帰関数を使って総重量を計算
    MC = calcMC(1); # 再帰関数を使って重心位置計算
    com = MC / M;
    
    return com

In [ ]:
# 重心位置の計算の例
calcCoM()

In [ ]:
# 順運動学計算 2022/1/18
def ForwardAllKinematics(j):

    if j == 0 : return;
    if j != 1 :
        mom = arr[j].mother;
        # position and orientation
        arr[j].p = np.matmul(arr[mom].R, arr[j].b) + arr[mom].p;
        arr[j].R = np.matmul(arr[mom].R, Rodrigues(arr[j].a, arr[j].q));
        
        # spatial velocity
        hw = np.matmul(arr[mom].R, arr[j].a);  # axis vector in world frame
        hv = np.cross(arr[j].p, hw);      # p_i x axis
        
        arr[j].hw = hw;                # store h1 and h2 for future use
        arr[j].hv = hv;
        
        arr[j].w  = arr[mom].w  + hw * arr[j].dq;
        arr[j].vo = arr[mom].vo + hv * arr[j].dq; 
        
        # spatial acceleration
        dhv = np.cross(arr[mom].w, hv) + np.cross(arr[mom].vo, hw);
        dhw = np.cross(arr[mom].w, hw);
        
        arr[j].dw  = arr[mom].dw  + dhw * arr[j].dq + hw * arr[j].ddq;
        arr[j].dvo = arr[mom].dvo + dhv * arr[j].dq + hv * arr[j].ddq;  
    
    ForwardAllKinematics(arr[j].sister);
    ForwardAllKinematics(arr[j].child);


In [ ]:
# 床からの外力計算 2022/1/18
def ExtForce(j):
    #w_c = uLINK(j).R * uLINK(j).c + uLINK(j).p;   # 重心位置
    #f = [0 0 -uLINK(j).m * G]';    # 重力
    f   = np.array([0., 0., 0.]);        # 床反力
    t   = np.array([0., 0., 0.]);        # モーメント
    Kf  = (1.0E+4);        # 床面の剛性[N/m]
    Df  = (1.0E+3);        # 床面の粘性[N/(m/s)]
    w_p = np.array([0., 0., 0.]);  

    for n in range(4): # 足裏の４点の位置をチェック0-3
    #if(isempty(arr[j].vertex) != 1) :
        w_p = np.matmul(arr[j].R, arr[j].vertex[n,:]); # ワールド座標へ回転
        pos = w_p + arr[j].p; #ワールド座標での位置
        if pos[2] < 0.0 :  # 支点は床面と接触している
            v = arr[j].vo + np.cross(arr[j].w, pos);  # ワールド座標の速度
            fc = np.array([-Df * v[0],  -Df * v[1], -Kf * pos[2] - Df * v[2]]);
            f = f + fc;
            t = t +  np.cross(pos, fc);

    f = f/4;
    t = t/4;

    return f, t

In [ ]:
#　逆運動学計算アルゴリズム　2022/5/29　修正
def InverseDynamics(j, Dtime):

    if j == 0 : 
        f=np.array([0,0,0]);
        t=np.array([0,0,0]);
        return f, t

    c = np.matmul(arr[j].R, arr[j].c) + arr[j].p;   # center of mass : wcと同じ ForwardDynamicsABM.cpp line 307
    # print("check inv ", np.matmul(arr[j].R, arr[j].R.T)) # check transform
    I = np.matmul(arr[j].R, np.matmul(arr[j].I, arr[j].R.T));  # inertia in world frame　： Iwと同じ ForwardDynamicsABM.cpp line 310
    c_hat = hat(c);
    # print("c_hat", c_hat)
    # print("c_hat.T", c_hat.T)
    I = I + arr[j].m * np.matmul(c_hat, c_hat.T);  # Iwwの計算になっている．ForwardDynamicsABM.cpp line 314
    arr[j].Iww = I; # Iwwの保存 2022/5/29
    arr[j].Iwv = arr[j].m * c_hat; # Iwvの保存　2022/5/29
    arr[j].Ivv = np.eye(3)*arr[j].m; # Ivvの保存　2022/5/29

    P = arr[j].m * (arr[j].vo + np.cross(arr[j].w,c));   # linear  momentum
    L = arr[j].m * np.cross(c, arr[j].vo) + np.matmul(I, arr[j].w); # angular momentum

    f0 = arr[j].m * (arr[j].dvo + np.cross(arr[j].dw,c))   + np.cross(arr[j].w, P);
    t0 = arr[j].m * np.cross(c, arr[j].dvo) + np.matmul(I, arr[j].dw) + np.cross(arr[j].vo, P) + np.cross(arr[j].w, L);


    # from child link
    [f1,t1] = InverseDynamics(arr[j].child, Dtime);
    f = f0 + f1;
    t = t0 + t1;

    # ここから外力項を追加　f -= link->F_ext(); 重心位置に加わる外力
    # 床と足裏の接触
    if j == 7 or j == 13 or j == 1: #右足先と左足先のリンク j==1を入れるとボディのvetexも床面との接触チェックをする
      [f3, t3]=ExtForce(j); #接触力の計算（ただのバネダンパ）
      f = f - f3; #符号に注意
      t = t - t3;

    if j != 1 : # BODY(ID=1)以外の計算
        arr[j].u = np.dot(arr[j].hv, f) + np.dot(arr[j].hw, t);  # joint driving force #ここは内積


    # return force to mother, with sisters
    [f2,t2] = InverseDynamics(arr[j].sister,Dtime);
    f = f + f2;
    t = t + t2;

    # 20220529 ForwardDynamicsABM.cpp line 333-334 のように各リンクにバイアス項保存
    arr[j].pf = f;
    arr[j].ptau = t;

    return f,t

In [ ]:
# 逆運動学計算の関数：単位ベクトル法 2022/2/21
def InvDyn(j):
    arr[1].dvo = np.array([0, 0, 0]);
    arr[1].dw  = np.array([0, 0, 0]);
    if j >= 1 and j <= 3 :
        arr[1].dvo[j-1] = 1; # BODYの加速度を1に設定
    elif j >= 4 and j <= 6:
        arr[1].dw[j-4] = 1;  # BODYの角加速度を1に設定
    
    arr[1].dvo = arr[1].dvo + np.array([0, 0, G]);  # ボディに加わる重力の効果を加える

    for n in range(1,len(arr)-1): # ボディ以下のリンクの計算
        if n == j-6:
            arr[n+1].ddq = 1.0;
        else:
            arr[n+1].ddq = 0.0;

    ForwardAllKinematics(1);
    [f,t] = InverseDynamics(1,Dtime);
    u = np.hstack([f,t]);
    for n in range(2,len(arr)):
        u=np.hstack([u,arr[n].u]);
    #u = [f,t,arr[1:].u];
    
    return u


In [ ]:
# ForwardDynamics
# 単位ベクトル法に基づく順動力学の計算
def ForwardDynamics(u_joint):
    nDoF = len(arr)-1+6-1; #18次元
    #print("nDoF ", nDoF);
    A = np.zeros((nDoF,nDoF));
    b = InvDyn(0);
    #print("b ", len(b))
    for n in range(1,nDoF):
        A[:,n-1] = InvDyn(n) - b;
    
    # モータ電機子慣性モーメントの効果
    #for n in range(7,nDoF):
    for n in range(6,nDoF-1):
        j = n-6+1+1;
        A[n,n] = A[n,n] + arr[j].Ir * arr[j].gr**2; # grはスカラ
    #print("len(A) ", len(A))

    # 加速度の計算
    #u  = [0, 0, 0, 0, 0, 0, u_joint(2:end)];
    #u = np.hstack([0, 0, 0, 0, 0, 0],u_joint[2:]);
    u = np.hstack([np.zeros(6),u_joint[1:]])
    #print("len(u) ", len(u))
    ddq = np.matmul(np.linalg.pinv(A), (-b + u));
    arr[1].dvo = ddq[0:3];
    #print("arr[1].dvo ",arr[1].dvo)
    arr[1].dw  = ddq[3:6];
    #print("arr[1].dw ",arr[1].dw)
    #print("len(arr) ", len(arr))
    for j in range(2,len(arr)):
        arr[j].ddq = ddq[j+6-2];
    


In [ ]:
# IntegrateEuler
def IntegrateEuler(j,Dtime):
    if j == 0 : return;
    if j == 1 :
        [arr[j].p, arr[j].R] = SE3exp(j, Dtime);
        #print("len(arr[j].vo ",  len(arr[j].vo))
        #print("len(arr[j].dvo ", len(arr[j].dvo))
        arr[j].vo = arr[j].vo + arr[j].dvo * Dtime;
        arr[j].w  = arr[j].w  + arr[j].dw * Dtime;
    else :
        arr[j].q  = arr[j].q  + arr[j].dq * Dtime;
        arr[j].dq = arr[j].dq + arr[j].ddq * Dtime;


    IntegrateEuler(arr[j].sister, Dtime);
    IntegrateEuler(arr[j].child, Dtime);


In [ ]:
# DrawAllJoints 全関節を線で描画する再帰関数．vertexがあるリンクはvertexを一筆書きで描画（適当．．．）
def DrawAllJoints(j):
    # radius    = 0.02;
    # len       = 0.06;
    # joint_col = 0;

    if j != 0 : 
        if hasattr(arr[j], 'vertex') : #vertexメンバ変数を持っているかのチェック
            #print("joint ",j);
            vert = np.matmul(arr[j].R, arr[j].vertex.T);
            for k in range(3) :# 1:3
                vert[k,:] = vert[k,:] + arr[j].p[k];#    adding x,y,z to all vertex
            ax.plot(vert[0,:],vert[1,:],vert[2,:],"o-", c='b')
    
        i = arr[j].mother;
        if i != 0:
            ax.plot([arr[i].p[0],arr[j].p[0]],[arr[i].p[1],arr[j].p[1]],[arr[i].p[2],arr[j].p[2]],"o-", c='b')

        DrawAllJoints(arr[j].child);
        DrawAllJoints(arr[j].sister);


In [ ]:
# biped robot simulation
#  単位ベクトル法による動力学シミュレーション
#  床面との接触反力は足の長方形板の頂点のめり込みを簡易的に計算しているだけ．
#　剛体の接触力計算については．．．いい方法があれば教えて下さい．

G = 9.8;  # 重力加速度

arr = [0]*14;
SetupRobot(arr);# ロボットの初期設定

# 胴体の空間速度・加速度の初期化
BODY=1;
arr[BODY].vo = np.array([0., 0., 0.]);
arr[BODY].w  = np.array([0., 0., 0.]);
arr[BODY].dvo = np.array([0., 0., 0.]);
arr[BODY].dw  = np.array([0., 0., 0.]);

# 関節トルク
u_joint = np.zeros(len(arr)-1); # 全関節トルクを0に設定
#u_joint[RLEG_J2] = -10.0;   # 右股関節ピッチ軸にトルクを与える　[Nm]

# シミュレーション設定
Dtime = 0.001; # 1msでシミュレーションしないと簡易な接触力計算では発散してしまう．ルンゲクッタにすればましになるか？
EndTime = 0.7; # 0.7秒だけのトルクゼロの崩れるシミュレーション．
time = EndTime//Dtime+1; # 刻み数：最後の１は時刻ゼロも含めるため
com_m = np.zeros((int(time),3)); # 重心位置の初期化 プロットに使う？？


for k in range(int(time)):
    ForwardDynamics(u_joint); # 順運動学計算
    IntegrateEuler(1,Dtime); # 順運動学で求めたトルクから全関節のパラメータ更新
    com = calcCoM;  # 重心位置計算
    #com_m[k,:] = com;
    if k%10==0:
        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'}) #ここで図を用意

        print(k)
        DrawAllJoints(1); # ロボットの簡易描画
        
        ax.set_xticks(np.linspace(-1.0, 1.0, 5)) # X軸の表示範囲
        ax.set_yticks(np.linspace(-1.0, 1.0, 5)) # Y軸の表示範囲
        ax.set_zticks(np.linspace(-0.5, 1.5, 5)) # Z軸の表示範囲
        plt.show();

In [ ]:
# biped robot simulation  アニメーション保存バージョン
#  単位ベクトル法による動力学シミュレーション
#  床面との接触反力は足の長方形板の頂点のめり込みを簡易的に計算しているだけ．
#　剛体の接触力計算については．．．いい方法があれば教えて下さい．

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

G = 9.8;  # 重力加速度

arr = [0]*14;
SetupRobot(arr);# ロボットの初期設定

# 胴体の空間速度・加速度の初期化
BODY=1;
arr[BODY].vo = np.array([0., 0., 0.]);
arr[BODY].w  = np.array([0., 0., 0.]);
arr[BODY].dvo = np.array([0., 0., 0.]);
arr[BODY].dw  = np.array([0., 0., 0.]);

# 関節トルク
u_joint = np.zeros(len(arr)-1); # 全関節トルクを0に設定
#u_joint[RLEG_J2] = -10.0;   # 右股関節ピッチ軸にトルクを与える　[Nm]

# シミュレーション設定
Dtime = 0.001; # 1msでシミュレーションしないと簡易な接触力計算では発散してしまう．ルンゲクッタにすればましになるか？
EndTime = 0.5; # 0.5秒だけのトルクゼロの崩れるシミュレーション．
time = EndTime//Dtime+1; # 刻み数：最後の１は時刻ゼロも含めるため
com_m = np.zeros((int(time),3)); # 重心位置の初期化 プロットに使う？？

fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'}); #ここで図を用意

ims = []; # 空のイメージ保存用の配列用意
n = 0;
for k in range(int(time)):
    ForwardDynamics(u_joint); # 順運動学計算
    IntegrateEuler(1,Dtime); # 順運動学で求めたトルクから全関節のパラメータ更新
    com = calcCoM;  # 重心位置計算
    #com_m[k,:] = com;
    if k%10==0:
        print(k)
        #fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'}); #ここで図を用意
        DrawAllJoints(1); # ロボットの簡易描画
        ax.set_xticks(np.linspace(-1.0, 1.0, 5)) # X軸の表示範囲
        ax.set_yticks(np.linspace(-1.0, 1.0, 5)) # Y軸の表示範囲
        ax.set_zticks(np.linspace(-0.5, 1.5, 5)) # Z軸の表示範囲
        fig.savefig('./biped/'+str(n)+'.png'.format(0, 0)); # bipedディレクトリを作成しておくこと．bipedディレクトリに図を順番に保存
        ax.cla();
        n+=1;
        frames = n;
        #print(frames);
 
from PIL import Image, ImageFilter
for n in range(frames):
    exec('a'+str(n)+'=Image.open("./biped/'+str(n)+'.png");');
    ims.append(eval('a'+str(n)));

ims[0].save('./biped_robot_sim.gif'.format(0,0), save_all=True, append_images=ims[1:], duration=100, loop=0);

In [ ]:
def clamp(n, smallest, largest):
    return max(smallest, min(n, largest))

In [88]:
# 機構慣性の計算 2022/5/15
def AB():

    #const int n = subBody->numLinks();
    nDoF = len(arr)-1+6-1; #18次元

    #for(int i = n-1; i >= 0; --i){  for k in range(int(time)):
    for i in range(nDoF, 0, -1):
        #auto link = subBody->link(i); //n番目のリンクを初期化
        #link->pf()   -= link->f_ext(); // 外力を先に足しておく：もとはゼロのはず
        #link->ptau() -= link->tau_ext(); // 外力を先に足しておく
        # pf, ptauはバイアス項らしい？

        # compute articulated inertia (Eq.(6.48) of Kajita's textbook)
        #for(DyLink* child = link->child(); child; child = child->sibling()){ //子供と兄弟リンクを2回ループするだけ 

        for k in range(2): # //0: 子供, 1:兄弟リンクを2回ループするだけ
            if k == 0:
                childsister = arr[k].child;
            elif k == 1:
                chidlsister = arr[k].sister;

            #if(child->isFreeJoint()){
            if childsister == 0:
                continue;
            elif hasattr(arr[childsister], 'isFixedJoint') and arr[childsister].isFixedJoint == 1: #isFixedJointメンバ変数を持っているかのチェック
                #if arr[childsister].isFixedJoint == 1:
                arr[i].Ivv += arr[childsister].Ivv;
                arr[i].Iwv += arr[childsister].Iwv;
                arr[i].Iww += arr[childsister].Iww;
            else:
                #### ここから6.48式の更新らしいが．．． ###
                #const Vector3 hhv_dd = child->hhv() / child->dd();
                #arr[i].hhv_dd = np.array([0, 0, 0]);#この初期化必要か？いらないならコメントアウト．
                hhv_dd = arr[childsister].hhv / arr[childsister].dd;#hhv/dd ただし，dd=s^T*I^A*s, hhv=I^A*s は 3x1の縦ベクトル
                hhw_dd = arr[childsister].hhw / arr[childsister].dd;#hhw/dd
                arr[i].Ivv += arr[chidlsister].Ivv - arr[chidlsister].hhv @ hhv_dd.T;#hhv*hhv_dd.tranで縦の3x1ベクトルと横の1x3ベクトルで３ｘ３の行列を生成．演算子は「@」
                arr[i].Iwv += arr[chidlsister].Iwv - arr[chidlsister].hhw @ hhv_dd.T;
                arr[i].Iww += arr[chidlsister].Iww - arr[chidlsister].hhw @ hhw_dd.T;
                #### ここまで6.48式の更新 ###
            
            # 式6.44の右辺第2項のバイアス項の計算？　P=I^s・\itaを計算しかしてない？運動量Pがpfで角運動量Lがptauのはず．
            arr[i].pf   += arr[childsister].Ivv * arr[childsister].cv + arr[childsister].Iwv.T * arr[childsister].cw + arr[childsister].pf;
            arr[i].ptau += arr[childsister].Iwv * arr[childsister].cv + arr[childsister].Iww * arr[childsister].cw + arr[childsister].ptau;

            if hasattr(arr[childsister], 'isFixedJoint') and arr[childsister].isFixedJoint == 1: #isFixedJointメンバ変数を持っているかのチェック
                uu_dd = arr[childsister].uu / arr[childsister].dd; #uu/dd
                arr[i].pf   += uu_dd * arr[childsister].hhv; #固定されているから理想の力がそのまま加わるということかな？
                arr[i].ptau += uu_dd * arr[childsister].hhw;


        #if(i > 0){
        if(i>1):
            #if(!link->isFixedJoint()){ #固定関節ではないとき
            print("i =", i)
            if arr[i].isFixedJoint != 1: #固定関節ではないとき
                #// hh = Ia * s  6.48式の第3項で使うためなのか？
                #I^A・sの更新：hhv(I^A・sの上)とhhw(I^A・sの下)とddを更新
                arr[i].hhv = arr[i].Ivv * arr[i].sv + arr[i].Iwv.T * arr[i].sw;
                arr[i].hhw = arr[i].Iwv * arr[i].sv + arr[i].Iww * arr[i].sw;
                #// dd = Ia * s * s^T この式がわからんs^T * Ia * sならわかるが．6.48式の第3項の時には使うが？上のddはs^T*I^A*sと思っていたけど．．．
                arr[i].dd = arr[i].sv.dot(arr[i].hhv) + arr[i].sw.dot(arr[i].hhw) + arr[i].Jm2; # ddとはなんだ？ Jm2とは？->/// Equivalent rotor inertia: n^2*Jm [kg.m^2] Link.hより
                #// uu = u - hh^T*c + s^T*pp このuuってなんだ？この式もわからん．hh^T*cは6.49の第2項を展開したものでS^T*I^A*\dot{\ita}
                # clampは第2引数下限，第2引数上限を与えて，超えていたら上下限の値を返す関数．
                arr[i].uu = clamp(arr[i].u, arr[i].u_lower, arr[i].u_upper) - (arr[i].hhv().dot(arr[i].cv) + arr[i].hhw.dot(arr[i].cw) + arr[i].sv().dot(arr[i].pf) + arr[i].sw.dot(arr[i].ptau)); #ここはバイアス項：式6.49のb_2だが，式6.46のを見たほうが良い．
                    ## uuは式6.49の分子か？uのupperの意味？？ddで割れば式6.49の関節の角加速度が求まる！
    
    #return u


In [ ]:
# for文の練習
nDoF = 18;
for i in range(nDoF, 0, -1):
    print(i)
for i in range(2):
    print(i)
a=np.array([[1,2],[3, 4]])
print(a.T);


In [81]:
# ForwardDynamics_ab
# Articurated Body Dynamicsを使った順動力学の計算
def ForwardDynamics_ab(u_joint):

    ForwardAllKinematics(1);
    [f,t] = InverseDynamics(1,Dtime);

    AB();

    # AB_phaze3
    # 加速度の計算
    #auto root = subBody->rootLink();
    #if(!root->isFreeJoint()){
    if arr[u_joint].isFixedJoint == 1 : #固定関節
        arr[u_joint].dvo =  [0., 0., 0.]; #root->dvo().setZero();
        arr[u_joint].dw = [0., 0., 0.]; #root->dw().setZero();
    else:
        # - | Ivv  trans(Iwv) | * | dvo | = | pf   |
        #   | Iwv     Iww     |   | dw  |   | ptau |

        #Eigen::Matrix<double, 6, 6> M;
        #M << root->Ivv(), root->Iwv().transpose(),
        #    root->Iwv(), root->Iww();
        
        #Eigen::Matrix<double, 6, 1> f;
        #f << root->pf(),
        #    root->ptau();
        #f *= -1.0;

        #Eigen::Matrix<double, 6, 1> a(M.colPivHouseholderQr().solve(f));

        #root->dvo() = a.head<3>();
        #root->dw() = a.tail<3>();
        SE3dynamics(u_joint);

    #const int n = subBody->numLinks();
    n = len(arr) - 1; # n = 13 
    for i in range(1,n): # (int i=1; i < n; ++i){ #1 から 14まで
        # auto link = subBody->link(i);
        # auto parent = link->parent();
        #if(link->isFixedJoint()){
        if arr[i].isFixedJoint == 1 : 
            arr[i].ddq = 0.0;
            arr[i].dvo = arr[arr[i].mother].dvo;
            arr[i].dw  = arr[arr[i].mother].dw; 
        else :
            #link->ddq() = (link->uu() - (link->hhv().dot(parent->dvo()) + link->hhw().dot(parent->dw()))) / link->dd();
            #link->dvo().noalias() = parent->dvo() + link->cv() + link->sv() * link->ddq();
            #link->dw().noalias()  = parent->dw()  + link->cw() + link->sw() * link->ddq();
            arr[i].ddq = (arr[i].uu - np.dot(arr[i].hhv, arr[arr[i].mother].dvo) + np.dot(arr[i].hhw, arr[arr[i].mother].dw)) / arr[i].dd;
            arr[i].dvo = arr[arr[i].mother].dvo + arr[i].cv + arr[i].sv * arr[i].ddq;
            arr[i].dw  = arr[arr[i].mother].dw  + arr[i].cw + arr[i].sw * arr[i].ddq;




In [ ]:
# 順運動学計算 2022/1/18 　ここにあるのは上のコピーで変更は何もしていない．
def ForwardAllKinematics(j):

    if j == 0 : return;
    if j != 1 :
        mom = arr[j].mother;
        # position and orientation
        arr[j].p = np.matmul(arr[mom].R, arr[j].b) + arr[mom].p;
        arr[j].R = np.matmul(arr[mom].R, Rodrigues(arr[j].a, arr[j].q));
        
        # spatial velocity
        hw = np.matmul(arr[mom].R, arr[j].a);  # axis vector in world frame
        hv = np.cross(arr[j].p, hw);      # p_i x axis
        
        arr[j].hw = hw;                # store h1 and h2 for future use
        arr[j].hv = hv;
        
        arr[j].w  = arr[mom].w  + hw * arr[j].dq;
        arr[j].vo = arr[mom].vo + hv * arr[j].dq; 
        
        # spatial acceleration
        dhv = np.cross(arr[mom].w, hv) + np.cross(arr[mom].vo, hw);
        dhw = np.cross(arr[mom].w, hw);
        
        arr[j].dw  = arr[mom].dw  + dhw * arr[j].dq + hw * arr[j].ddq;
        arr[j].dvo = arr[mom].dvo + dhv * arr[j].dq + hv * arr[j].ddq;  
    
    ForwardAllKinematics(arr[j].sister);
    ForwardAllKinematics(arr[j].child);

In [89]:
# biped robot simulation
#  Articulated Body Dynamicsを使ったシミュレーション
G = 9.8;  # 重力加速度

arr = [0]*14;
SetupRobot(arr);# ロボットの初期設定

# 胴体の空間速度・加速度の初期化
BODY=1;
arr[BODY].vo = np.array([0., 0., 0.]);
arr[BODY].w  = np.array([0., 0., 0.]);
arr[BODY].dvo = np.array([0., 0., 0.]);
arr[BODY].dw  = np.array([0., 0., 0.]);

# 関節トルク
u_joint = np.zeros(len(arr)-1); # 全関節トルクを0に設定
#u_joint[RLEG_J2] = -10.0;   # 右股関節ピッチ軸にトルクを与える　[Nm]

# シミュレーション設定
Dtime = 0.001; # 1msでシミュレーションしないと簡易な接触力計算では発散してしまう．ルンゲクッタにすればましになるか？
EndTime = 0.7; # 0.7秒だけのトルクゼロの崩れるシミュレーション．
time = EndTime//Dtime+1; # 刻み数：最後の１は時刻ゼロも含めるため
com_m = np.zeros((int(time),3)); # 重心位置の初期化 プロットに使う？？

number = len(arr);
for i in range(1,number):
    arr[i].isFixedJoint = 0;

for k in range(int(time)):
    ForwardDynamics_ab(u_joint); # 順運動学計算
    IntegrateEuler(1,Dtime); # 順運動学で求めたトルクから全関節のパラメータ更新
    com = calcCoM;  # 重心位置計算
    #com_m[k,:] = com;
    if k%10==0:
        fig, ax = plt.subplots(figsize=(6, 6), subplot_kw={'projection' : '3d'}) #ここで図を用意

        print(k)
        DrawAllJoints(1); # ロボットの簡易描画
        
        ax.set_xticks(np.linspace(-1.0, 1.0, 5)) # X軸の表示範囲
        ax.set_yticks(np.linspace(-1.0, 1.0, 5)) # Y軸の表示範囲
        ax.set_zticks(np.linspace(-0.5, 1.5, 5)) # Z軸の表示範囲
        plt.show();

i = 18


IndexError: list index out of range